# Scipt para buscar títulos da base de dados BB para a base de dados de CPB.


## 1) Carregamento e importação de bibliotecas e bases de dados 

In [1]:
#Importa bibliotecas importantes
import pandas as pd
import numpy as np
from IPython.display import display, HTML
import re
import os
import unidecode
import string
import nltk
import warnings
#nltk.download('punkt')
from nltk.tokenize import word_tokenize
from pandas import ExcelWriter
from tqdm import tqdm


In [2]:
#carrega Matchs de 2024
matchs_2024 = pd.read_excel('Bases/Matchs até 2024.xlsx', engine='openpyxl')

In [3]:
# Carregando os datasets mais recentes da Ancine
Base_SAD = pd.read_excel('Bases/Obras Ancine.xlsx', engine='openpyxl')


In [4]:
#Carregando base BB mais recente de 2025
Base_BB_import = pd.read_excel('Bases/BB Media - 2025-08-28 - Brazil - ALL PLATFORMS.xlsx', engine='openpyxl')


In [5]:
#Carregando base BB já percorrida
Base_BB_2024 =  pd.read_excel('Bases/Base BB 2024 inicial.xlsx', engine='openpyxl')


In [6]:
# ================================================================
# Resumo das bases carregadas
# ================================================================
bases = {
    'matchs_2024': matchs_2024,
    'Base_SAD': Base_SAD,
    'Base_BB': Base_BB_import,
    'Base_BB_2024': Base_BB_2024
}

print("Resumo das bases carregadas:\n")
for nome, df in bases.items():
    print(f"{nome:<15}: {len(df):>10,} linhas | {df.shape[1]:>3} colunas")


Resumo das bases carregadas:

matchs_2024    :      8,515 linhas |   6 colunas
Base_SAD       :     59,911 linhas |  12 colunas
Base_BB        :    378,618 linhas |  46 colunas
Base_BB_2024   :    535,758 linhas |  30 colunas


## 1.1) Limpeza de registros desnecessários na base da BB

In [7]:
# Remover duplicatas dentro de Base_BB_1_filtrada, mantendo apenas o primeiro registro de cada UID
Base_BB_inicial = Base_BB_import.drop_duplicates(subset='BB UID')

# Contador de linhas final
print(f"Quantidade de linhas em Base_BB_inicial: {len(Base_BB_inicial)}")


Quantidade de linhas em Base_BB_inicial: 247692


In [8]:
# Filtrar Base_BB removendo registros já existentes em Base_BB_2024
Base_BB_inicial = Base_BB_inicial[~Base_BB_inicial['BB UID'].isin(Base_BB_2024['UID'])].copy()

# Opcional: resetar o índice para limpeza visual
Base_BB_inicial.reset_index(drop=True, inplace=True)

print(f"Registros restantes em Base_BB_inicial: {len(Base_BB_inicial)}")


Registros restantes em Base_BB_inicial: 119267


In [9]:
# ================================================================
# Filtrando plataformas indesejadas da base Base_BB_inicial
# ================================================================

PLATAFORMAS_EXCLUIR = [
    'Archivio Luce', 'NBA League Pass', 'iQIYI', 'Zee5', 'FIFA+', 'Qello Concerts', 'ShemarooMe',
    'FlixOlé', 'Rakuten Viki', 'OnDemandKorea', 'Simply South', 'Digital Concert Hall', 'IWantTFC',
    'RT en Español', 'KOCOWA+', 'CINE.AR PLAY', 'IFI Archive Player', 'Hoichoi', 'Selecta TV',
    'Met Opera on Demand', 'Reel Short', 'American Indian Film Gallery', 'TVN Play', 'Teatrix',
    'Retina Latina', 'Demand Africa', 'DAZN', 'HispanTV', 'HENRI', 'Anime Onegai', 'ALTBalaji',
    'Digital Theatre', 'F1 TV'
]

# Contagem inicial
total_antes = len(Base_BB_inicial)

# Filtrando registros cujo campo 'Platform Name' está na lista
Base_BB_inicial = Base_BB_inicial[~Base_BB_inicial['Platform Name'].isin(PLATAFORMAS_EXCLUIR)].copy()

# Resetando índice
Base_BB_inicial.reset_index(drop=True, inplace=True)

# Contagem final e resumo
total_depois = len(Base_BB_inicial)
removidos = total_antes - total_depois

print(f"Total antes do filtro: {total_antes:,}")
print(f"Total após o filtro : {total_depois:,}")
print(f"Registros removidos : {removidos:,}")


Total antes do filtro: 119,267
Total após o filtro : 39,417
Registros removidos : 79,850


## 1.2) Resumo Estatístico dos dados

In [10]:
# ================================================================
# Resumo estatístico e estrutural das bases Base_SAD e Base_BB_inicial
# ================================================================

def resumo_dataframe(df, nome_df):
    """
    Exibe resumo de atributos e estatísticas descritivas de um DataFrame
    em formato amigável e legível no JupyterLab.
    """
    display(HTML(f"<h3 style='color:#2a5599;'>📊 Resumo da base <code>{nome_df}</code></h3>"))

    # === 1️⃣ Estrutura e tipos ===
    info_df = pd.DataFrame({
        "Coluna": df.columns,
        "Tipo de Dado": [str(df[col].dtype) for col in df.columns],
        "Valores Não Nulos": [df[col].notna().sum() for col in df.columns],
        "Valores Nulos": [df[col].isna().sum() for col in df.columns],
        "% Nulos": [df[col].isna().mean() * 100 for col in df.columns]
    })

    styled_info = (
        info_df.style
        .format({'% Nulos': '{:.2f}%'})
        .set_table_styles([
            {'selector': 'th', 'props': [('background-color', '#f0f0f0'), ('text-align', 'center')]},
            {'selector': 'td', 'props': [('padding', '6px 12px')]},
        ])
        .set_properties(subset=['Coluna'], **{'text-align': 'left'})
        .set_properties(subset=['Tipo de Dado'], **{'text-align': 'center'})
        .set_properties(subset=['Valores Não Nulos', 'Valores Nulos', '% Nulos'], **{'text-align': 'right'})
        .hide(axis="index")
    )
    display(HTML("<h4>📋 Estrutura e Qualidade dos Dados</h4>"))
    display(styled_info)

    # === 2️⃣ Estatísticas descritivas ===
    desc = df.describe(include='all').transpose().fillna("")
    styled_desc = (
        desc.style
        .set_table_styles([
            {'selector': 'th', 'props': [('background-color', '#f0f0f0'), ('text-align', 'center')]},
            {'selector': 'td', 'props': [('padding', '6px 12px')]},
        ])
        .set_properties(**{'text-align': 'right'})
    )
    display(HTML("<h4>📈 Resumo Estatístico</h4>"))
    display(styled_desc)
    display(HTML("<hr style='margin:25px 0;'>"))

# ================================================================
# Aplicação da função às suas bases
# ================================================================
resumo_dataframe(Base_SAD, "Base_SAD")
resumo_dataframe(Base_BB_inicial, "Base_BB_inicial")


Coluna,Tipo de Dado,Valores Não Nulos,Valores Nulos,% Nulos
ID_OBRA,object,59911,0,0.00%
Título Original,object,59910,1,0.00%
Título Alternativo,object,1,59910,100.00%
Classificação,object,59911,0,0.00%
Gênero,object,59911,0,0.00%
Organização Temporal,object,59911,0,0.00%
Duração,float64,59837,74,0.12%
Nº de Episódios,float64,16426,43485,72.58%
Ano de Produção,object,59909,2,0.00%
Produtora,object,59900,11,0.02%


C:\Users\Guilherme\AppData\Local\Temp\ipykernel_10380\904775675.py:37: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  desc = df.describe(include='all').transpose().fillna("")


,count,unique,top,freq,mean,std,min,25%,50%,75%,max
ID_OBRA,59911.000000,59856,B2000397700000,3,,,,,,,
Título Original,59910.000000,55591,VIDEOAULAS #NEM1PRATRAS,13,,,,,,,
Título Alternativo,1.000000,1,SE EU FOSSE VOCÊ,1,,,,,,,
Classificação,59911.000000,4,BRASILEIRA INDEPENDENTE CONSTITUINTE DE ESPAÇO QUALIFICADO,30880,,,,,,,
Gênero,59911.000000,11,FICÇÃO,17686,,,,,,,
Organização Temporal,59911.000000,5,NÃO SERIADA,47791,,,,,,,
Duração,59837.000000,,,,267.925638,3455.872003,0.130000,5.750000,27.000000,90.000000,470400.000000
Nº de Episódios,16426.000000,,,,23.952332,91.722806,1.000000,1.000000,8.000000,15.000000,3480.000000
Ano de Produção,59909.000000,315,2023,2945,,,,,,,
Produtora,59900.000000,19252,GLOBO COMUNICACAO E PARTICIPACOES S/A,2343,,,,,,,


Coluna,Tipo de Dado,Valores Não Nulos,Valores Nulos,% Nulos
Unnamed: 0,int64,39417,0,0.00%
BB UID,object,39417,0,0.00%
Platform Content ID,object,39417,0,0.00%
BB Hash Unique,object,39417,0,0.00%
Platform Name,object,39417,0,0.00%
Platform Country,object,39417,0,0.00%
Package,object,39417,0,0.00%
Platform Title,object,39417,0,0.00%
Type,object,39417,0,0.00%
Deeplink,object,39417,0,0.00%


C:\Users\Guilherme\AppData\Local\Temp\ipykernel_10380\904775675.py:37: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  desc = df.describe(include='all').transpose().fillna("")


,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Unnamed: 0,39417.000000,,,,175863.559150,108471.144417,0.000000,104226.000000,168746.000000,242147.000000,373787.000000
BB UID,39417.000000,39417,bcba58f6d60b0a071c777c8b5f4df4f0,1,,,,,,,
Platform Content ID,39417.000000,39199.000000,1740.000000,3.000000,,,,,,,
BB Hash Unique,39417.000000,39417,c96dd8d4d65fa5545b2f9d68d996423b,1,,,,,,,
Platform Name,39417.000000,90,Amazon Prime Video,6335,,,,,,,
Platform Country,39417.000000,1,BR,39417,,,,,,,
Package,39417.000000,4,Subscription,17656,,,,,,,
Platform Title,39417.000000,37853,TREINO DA SELEÇÃO BRASILEIRA,8,,,,,,,
Type,39417.000000,2,Movie,34187,,,,,,,
Deeplink,39417.000000,39297,https://www.cinehumbertomauromais.com,80,,,,,,,


In [11]:
# ================================================================
# Amostra das primeiras linhas (head) das bases Base_SAD e Base_BB_inicial
# ================================================================

def exibir_head(df, nome_df, linhas=10):
    """
    Exibe as primeiras linhas de um DataFrame (head) com formatação
    legível no JupyterLab, no estilo de tabela do Colab.
    """
    display(HTML(f"<h3 style='color:#2a5599;'>🔎 Amostra das {linhas} primeiras linhas da base <code>{nome_df}</code></h3>"))

    styled_head = (
        df.head(linhas)
        .style
        .set_table_styles([
            {'selector': 'th', 'props': [('background-color', '#f0f0f0'), ('text-align', 'center')]},
            {'selector': 'td', 'props': [('padding', '6px 12px')]},
        ])
    )

    display(styled_head)
    display(HTML("<hr style='margin:25px 0;'>"))

# ================================================================
# Aplicação da função às duas bases
# ================================================================
exibir_head(Base_SAD, "Base_SAD", linhas=10)
exibir_head(Base_BB_inicial, "Base_BB_inicial", linhas=10)


,ID_OBRA,Título Original,Título Alternativo,Classificação,Gênero,Organização Temporal,Duração,Nº de Episódios,Ano de Produção,Produtora,Diretor,País de Origem
0,B0200000400000,SEJA O QUE DEUS QUISER,nan,BRASILEIRA INDEPENDENTE CONSTITUINTE DE ESPAÇO QUALIFICADO,FICÇÃO,NÃO SERIADA,90.000000,nan,2002,BRASIL FILMES LTDA | CINEMA BRASIL DIGITAL - ESCRITÓRIO DE PLANEJ. EMPR. AUD. LTDA | DISTRIBUIDORA DE FILMES S/A RIOFILME,MURILO NAVARRO DE SALLES,BR
1,B0200000600000,ROCINHA BRASIL77,nan,NÃO INFORMADO,NÃO CLASSIFICADA,NÃO SERIADA,19.000000,nan,1977,CONCEITO EVENTOS E PROMOÇÕES LTDA,SERGIO PÉO,BR
2,B0200000700000,ROCINHA 76/96,nan,NÃO INFORMADO,NÃO CLASSIFICADA,NÃO SERIADA,4.000000,nan,1996,CONCEITO EVENTOS E PROMOÇÕES LTDA,SERGIO PÉO,BR
3,B0200000800000,NANDERU - PANORÂMICA TUPINAMBÁ,nan,BRASILEIRA INDEPENDENTE CONSTITUINTE DE ESPAÇO QUALIFICADO,DOCUMENTÁRIO,NÃO SERIADA,9.000000,nan,1991,CONCEITO EVENTOS E PROMOÇÕES LTDA,SERGIO CASEMIRO JUCA DOS SANTOS,BR
4,B0200000900000,ABC BRASIL,nan,BRASILEIRA INDEPENDENTE CONSTITUINTE DE ESPAÇO QUALIFICADO,DOCUMENTÁRIO,NÃO SERIADA,18.520000,nan,1980,CONCEITO EVENTOS E PROMOÇÕES LTDA,SERGIO CASEMIRO JUCA DOS SANTOS,BR
5,B0200001000000,DURVAL DISCOS,nan,BRASILEIRA INDEPENDENTE CONSTITUINTE DE ESPAÇO QUALIFICADO,FICÇÃO,NÃO SERIADA,96.000000,nan,2001,DEZENOVE SOM E IMAGENS PRODUÇÕES LTDA - EPP | ÁFRICA FILMES LTDA,ANA LUIZA MACHADO DA SILVA MUYLAERT,BR
6,B0200001200000,RUA SEIS SEM NUMERO,nan,BRASILEIRA CONSTITUINTE DE ESPAÇO QUALIFICADO,FICÇÃO,NÃO SERIADA,103.000000,nan,2002,FILMOCENTRO SONIDO | OESTE FILMES BRASILEIROS LTDA-BAIXADO,JOAO BATISTA MORAES DE ANDRADE,"BR, CL"
7,B0200001300000,DESMUNDO,nan,BRASILEIRA CONSTITUINTE DE ESPAÇO QUALIFICADO,FICÇÃO,NÃO SERIADA,100.000000,nan,2001,A.F. CINEMA E VÍDEO LTDA | CONTINENTALFILMES | LABO CINE DO BRASIL LTDA,ALAIN FRESNOT,"BR, PT"
8,B0300001600000,DEUS É BRASILEIRO,nan,BRASILEIRA CONSTITUINTE DE ESPAÇO QUALIFICADO,FICÇÃO,NÃO SERIADA,110.000000,nan,2001,RIO VERMELHO FILMES LTDA,CARLOS JOSE FONTES DIEGUES,BR
9,B0300001700000,BURNING THE HOUSE,nan,NÃO INFORMADO,NÃO CLASSIFICADA,NÃO SERIADA,3.000000,nan,2002,O2 FILMES CURTOS LTDA,HANK LEVINE,BR


,Unnamed: 0,BB UID,Platform Content ID,BB Hash Unique,Platform Name,Platform Country,Package,Platform Title,Type,Deeplink,Seasons,Episodes,Season Numbers,Channel,Is Original,Is Exclusive,Platform Year,Platform Duration,Platform Rating,Platform Provider,Platform Genres,Platform Cast,Platform Directors,BB Coverage (In Days),BB Age,BB Cast,BB Countries,BB Directors,BB Duration,BB Genres,BB Keywords,BB Languages,BB Original Title,BB Primary Company,BB Primary Country,BB Primary Genre,BB Production Companies,BB Score,BB Scripted,BB Title,BB Year,BB Votes,IMDb ID,TMDB ID,TVDB ID,EIDR ID
0,0,bcba58f6d60b0a071c777c8b5f4df4f0,3d9e437e1879d37d8f4655b089f8d084,c96dd8d4d65fa5545b2f9d68d996423b,99 Media,BR,Free/Free with Ads,"Une journée dans la vie d'Elisa Carrillo Cabrera, danseuse étoile",Movie,https://www.99.media/fr/une-journee-dans-la-vie-delisa-carrillo-cabrera-danseuse-etoile/,nan,nan,nan,nan,No,No,nan,nan,nan,nan,nan,nan,Veronika Pokoptceva,49.000000,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,1,9bc637729b1d4946e22bdb0a4f79e335,e27619bb37bde6b77d84c782f64d23fb,a17ee5aa0be0c88dc5f7da29e5797ff7,99 Media,BR,Free/Free with Ads,"Volda, petite ville norvégienne, est-elle prête pour sa première Pride ?",Movie,https://www.99.media/fr/volda-petite-ville-rurale-norvegienne-est-elle-prete-pour-sa-toute-premiere-marche-des-fiertes/,nan,nan,nan,nan,No,No,nan,nan,nan,nan,nan,nan,Differ Media,57.000000,N/D,nan,nan,Differ Media,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,"Volda, petite ville norvégienne, est-elle prête pour sa première",nan,nan,nan,nan,nan,nan
2,2,ff6ec432972002dee46b4302b972ad8a,1bb766c1b6a95b9261d592740a4a7666,0906f056aee0a48bca7a90ffccbbcce0,99 Media,BR,Free/Free with Ads,"Pour Vova et Roma, la guerre est tout sauf un jeu en Ukraine",Movie,https://www.99.media/fr/pour-vova-et-roma-la-guerre-est-tout-sauf-un-jeu-en-ukraine/,nan,nan,nan,nan,No,No,nan,nan,nan,nan,nan,nan,I SEE YOU,140.000000,N/D,nan,nan,I SEE YOU,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,"Pour Vova et Roma, la guerre est tout sauf un jeu en Ukraine",nan,nan,nan,nan,nan,nan
3,3,882ede3bf128a44bae419ba155e43d16,75d56b37da8e9decee3d71f039836f96,ce6e6069736c385e6be00300cb699ff7,99 Media,BR,Free/Free with Ads,"Jorge, un DJ aux ""pieds tordus""",Movie,https://www.99.media/fr/jorge-un-dj-aux-pieds-tordus/,nan,nan,nan,nan,No,No,nan,nan,nan,nan,nan,nan,lospiestorcidos,169.000000,N/D,nan,nan,lospiestorcidos,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,"Jorge, un DJ aux ""pieds tordus""",nan,nan,nan,nan,nan,nan
4,4,228ccf03f6321b5340b3a7bc0c498f35,f02d7c9c4aebd7553d9a402865d35c8e,e20dc92ef78cde16222749262aea17b3,99 Media,BR,Free/Free with Ads,"Léandre, une histoire de détermination",Movie,https://www.99.media/fr/leandre-une-histoire-de-determination/,nan,nan,nan,nan,No,No,nan,nan,nan,nan,nan,nan,Matthieu Perret,230.000000,N/D,nan,nan,Léandre Boyer,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,"Léandre, une histoire de détermination",nan,nan,nan,nan,nan,nan
5,5,5c03ba31df28d9d290bb974a226f08e1,a7c14a02278ad1c421fb14b7e23ab3be,103b66f428bdb564eb199878fa744da2,99 Media,BR,Free/Free with Ads,Zvonko attend le train,Movie,https://www.99.media/fr/zvonko-attend-le-train/,nan,nan,nan,nan,No,No,nan,nan,nan,nan,nan,nan,Melita Vrsaljko,349.000000,N/D,nan,nan,Melita Vrsaljko,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,Zvonko attend le train,nan,nan,nan,nan,nan,nan
6,6,ad83ce65126e9bc26cca5c2ff5ee6cda,6a30201de68735314a9d144ea338a007,6171d0b16ab0f31a078d3b1de19a21c4,99 Media,BR,Free/Free with Ads,"En Finlande, tondre la pelouse, c'est... toute une histoire",Movie,https://www.99.media/fr/en-finlande-tondre-la-pelouse-cest-toute-une-histoire/,nan,nan,nan,nan,No,No,nan,nan,nan,nan,nan,nan,Otso Alanko,349.000000,N/D,nan,nan,Otso Alanko,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,"En Finlande, tondre la pelouse, c'est... toute une histoire",nan,nan,nan,nan,nan,nan
7,16,a463770eeaab3b49033e89b5f0cfc8aa,189728389949abbe516c26fd5621ffc1,339e83cddb3672

## 2) Pré-processamento e limpeza inicial de dados e criação de dataframes separados

## 2.1) Trimming do dataframe da BB

In [12]:
# ================================================================
# Selecionar apenas as colunas de interesse da base Base_BB_inicial
# ================================================================

cols_interesse = [
    'BB UID',    'Platform Title',    'Type',    'Deeplink',    'Seasons',    'Episodes',    'Season Numbers',    'BB Cast',    'BB Countries',    'BB Directors',    'BB Duration',    'BB Languages',
    'BB Original Title',    'BB Primary Company',    'BB Primary Country',    'BB Primary Genre',    'BB Production Companies',    'BB Title',    'BB Year',    'IMDb ID']

# Mantém somente as colunas listadas
Base_BB_trim = Base_BB_inicial[cols_interesse].copy()

# Exibe confirmação e amostra
display(HTML(f"<h4 style='color:#2a5599;'>📋 Base_BB_trim criada com {len(Base_BB_trim):,} registros e {len(Base_BB_trim.columns)} colunas.</h4>"))
display(Base_BB_trim.sample(5).style.set_table_styles([
    {'selector': 'th', 'props': [('background-color', '#f0f0f0'), ('text-align', 'center')]},
    {'selector': 'td', 'props': [('padding', '6px 12px')]}
]))


,BB UID,Platform Title,Type,Deeplink,Seasons,Episodes,Season Numbers,BB Cast,BB Countries,BB Directors,BB Duration,BB Languages,BB Original Title,BB Primary Company,BB Primary Country,BB Primary Genre,BB Production Companies,BB Title,BB Year,IMDb ID
31291,f525577f699117f8da18f8afed4ac03b,Ordinary Man,Movie,https://watch.plex.tv/movie/ordinary-man-2022-1,nan,nan,nan,nan,nan,nan,89.000000,nan,nan,nan,nan,nan,nan,nan,2022.000000,nan
11186,a7f33c77db5521f83d6f91efcac4350a,CHARLES E MEDGARD EVERS,Movie,http://bcc.cinemateca.org.br/filmes/447542,nan,nan,nan,nan,nan,nan,1.000000,nan,nan,nan,nan,nan,nan,CHARLES E MEDGARD EVERS,1968.000000,nan
19673,ea2cd100e87ecd330a4deb676a188481,A Vampira Nua,Movie,https://darkflixplus.com/vod/detail/20389,nan,nan,nan,"Olivier Rollin, Maurice Lemaître, Caroline Cartier, Ly Lestrong, Bernard Musson, Jean Aron, Ursule Pauly, Catherine Castel, Marie-Pierre Castel, Michel Delahaye, Pascal Fardoulis, Paul Bisciglia, Nicole Isimat","DE, FR",Jean Rollin,85.000000,"french, fr",The Nude Vampire,Les Films Abc,FR,Sci-fi,"Tigon British Film Productions, Box Office International, Les Films Abc, Les Distributeurs Associés, Redemption Films, L.c.j. Editions & Productions",The Nude Vampire,1970.000000,tt0065168
5212,70f9511184f6c745f879bcd74d5501ec,"Black Label Society - Boozed, Bruised and Broken Boned",Movie,https://app.primevideo.com/detail?gti=amzn1.dv.gti.feba6552-680c-f169-4463-de41ca35072e,nan,nan,nan,"Zakk Wylde, Nick Catanese, Craig Nunenmacher, Robert Trujillo","US, XX",Kenneth Botelho,120.000000,"english, en","Black Label Society - Boozed, Broozed & Broken-Boned",nan,US,Music,nan,"Black Label Society - Boozed, Broozed & Broken-Boned",2003.000000,tt0385552
6662,727bb2f6b5a3d5da3c0598c8196bba5c,MAHMUNDI EXCLUSIVA,Movie,https://arte1play.com.br/colecoes/arte1-em-movimento-pilulas-CdIlYV8voaM/mahmundi-exclusiva-mAx35kK7ARg,nan,nan,nan,nan,nan,Gisele Kato,4.000000,nan,nan,nan,nan,nan,nan,MAHMUNDI EXCLUSIVA,2023.000000,nan


In [13]:
Base_BB_trim.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39417 entries, 0 to 39416
Data columns (total 20 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   BB UID                   39417 non-null  object 
 1   Platform Title           39417 non-null  object 
 2   Type                     39417 non-null  object 
 3   Deeplink                 39417 non-null  object 
 4   Seasons                  4299 non-null   float64
 5   Episodes                 4299 non-null   float64
 6   Season Numbers           4299 non-null   object 
 7   BB Cast                  9881 non-null   object 
 8   BB Countries             10434 non-null  object 
 9   BB Directors             19244 non-null  object 
 10  BB Duration              29814 non-null  float64
 11  BB Languages             10099 non-null  object 
 12  BB Original Title        10831 non-null  object 
 13  BB Primary Company       8986 non-null   object 
 14  BB Primary Country    

In [14]:
# Exportando o dataframe Base_BB para o formato Excel
Base_BB_trim.to_excel('Bases/Base_BB_inicial_cortada.xlsx', index=False, engine='openpyxl')


In [15]:
# Recuperando o dataframe Base_BB para economizar tempo de carregamento

Base_BB = pd.read_excel('Bases/Base_BB_inicial_cortada.xlsx', engine='openpyxl')

### 2.2) Preenchimento de vazios SAD

In [16]:
# ================================================================
# 2.2) Preenchimento e ajustes de colunas na Base_SAD
# ================================================================

# 1️⃣ Preencher vazios nos campos textuais com NaN explícito
Base_SAD['Título Original'] = Base_SAD['Título Original'].fillna(np.nan)
Base_SAD['Produtora'] = Base_SAD['Produtora'].fillna(np.nan)
Base_SAD['Diretor'] = Base_SAD['Diretor'].fillna(np.nan)

# 2️⃣ Renomear coluna 'Duração' para 'Duração Obra'
if 'Duração' in Base_SAD.columns:
    Base_SAD = Base_SAD.rename(columns={'Duração': 'Duração Obra'})

# 3️⃣ Preencher vazios em 'Duração Obra' com zero
if 'Duração Obra' in Base_SAD.columns:
    Base_SAD['Duração Obra'] = Base_SAD['Duração Obra'].fillna(0)

# 4️⃣ Renomear coluna 'ID_OBRA' para 'Nº CPB'
if 'ID_OBRA' in Base_SAD.columns:
    Base_SAD = Base_SAD.rename(columns={'ID_OBRA': 'Nº CPB'})

# 5️⃣ Mensagem de confirmação
display(HTML(
    "<h4 style='color:#2a5599;'>✅ Preenchimento e renomeações concluídos com sucesso para <code>Base_SAD</code>.</h4>"
))

display(Base_SAD.sample(10).style.set_table_styles([
    {'selector': 'th', 'props': [('background-color', '#f0f0f0'), ('text-align', 'center')]},
    {'selector': 'td', 'props': [('padding', '6px 12px')]}
]))

,Nº CPB,Título Original,Título Alternativo,Classificação,Gênero,Organização Temporal,Duração Obra,Nº de Episódios,Ano de Produção,Produtora,Diretor,País de Origem
20556,B1402173300000,VIAJOLÂNDIA - VIAJO POR PORTO ALEGRE,nan,BRASILEIRA INDEPENDENTE CONSTITUINTE DE ESPAÇO QUALIFICADO,DOCUMENTÁRIO,SERIADA,32.730000,7.000000,2012 A 2012,VULCANA CINEMA LTDA,"FREDERICO AUGUSTO RUAS DOS SANTOS, FELIPE TRINDADE RECH",BR
23467,B1500207300000,ALCIONE - ESTRANHA LOUCURA - (EXTRAÍDO MÚSICA BOA),nan,BRASILEIRA CONSTITUINTE DE ESPAÇO QUALIFICADO,VÍDEOMUSICAL,NÃO SERIADA,2.520000,nan,2014,GLOBOSAT PROGRAMADORA LTDA (BAIXADA),PEDRO HENRIQUE SECCHIN SILVA,BR
9830,B0901068200000,ESTAÇÃO SAÚDE,nan,BRASILEIRA CONSTITUINTE DE ESPAÇO QUALIFICADO,DOCUMENTÁRIO,SERIADA,1980.000000,66.000000,2009 A 2009,PANORÂMICA COMUNICAÇÃO LTDA,RODRIGO AMADO DE ALBUQUERQUE MONTENEGRO,nan
56868,B2400521600000,CULTURAS DA TERRA – BIOMA AMAZÔNIA,nan,BRASILEIRA INDEPENDENTE CONSTITUINTE DE ESPAÇO QUALIFICADO,DOCUMENTÁRIO,NÃO SERIADA,53.150000,nan,2024,LEANDRO S. VERZINI PRODUÇÕES CINEMATOGRAFICAS,LEANDRO DOS SANTOS VERZINI,BR
15048,B1201595300000,EMBOLADA DA VIDA INTEIRA,nan,BRASILEIRA CONSTITUINTE DE ESPAÇO QUALIFICADO,DOCUMENTÁRIO,NÃO SERIADA,14.370000,nan,1994,EMVIDEO EVENTOS AUDIOVISUAIS LTDA,FRANCISCO DE PAULA CASTRO NETO,BR
19004,B1302009000000,ASSASSINATO NA CASA 436,nan,BRASILEIRA INDEPENDENTE CONSTITUINTE DE ESPAÇO QUALIFICADO,FICÇÃO,NÃO SERIADA,2.250000,nan,2011,WERGER PRODUÇÕES ARTÍSTICAS LTDA,LEO BOSCO GRIGGI PEDROSA,BR
37996,B1900321700000,AMANHECER - SELVA,nan,BRASILEIRA INDEPENDENTE CONSTITUINTE DE ESPAÇO QUALIFICADO,VÍDEOMUSICAL,NÃO SERIADA,3.450000,nan,2019,ELEMENTO CULTURAL PRODUCOES ARTISTICAS LTDA,FILIPE GUIMARAES NEVARES,BR
22483,B1402741400000,MISSÃO DE PAZ,nan,BRASILEIRA INDEPENDENTE CONSTITUINTE DE ESPAÇO QUALIFICADO,VÍDEOMUSICAL,NÃO SERIADA,4.400000,nan,2013,PAULO SERGIO QUERINO JUNIOR,PAULO SERGIO QUERINO JUNIOR,BR
34495,B1800290600000,ESTADO-VIOLÊNCIA,nan,BRASILEIRA INDEPENDENTE CONSTITUINTE DE ESPAÇO QUALIFICADO,ANIMAÇÃO,NÃO SERIADA,5.300000,nan,2016,RAFAEL GHIRALDELLI DA SILVA,RAFAEL GHIRALDELLI DA SILVA,BR
42727,B2100001800000,BOTA PRA TOCAR TIM MAIA,nan,BRASILEIRA CONSTITUINTE DE ESPAÇO QUALIFICADO,VÍDEOMUSICAL,NÃO SERIADA,3.980000,nan,2021,BRUNA DANTAS ALVES PRODUCOES,BRUNO BIAR MURTINHO BRAGA,BR


### 2.3) Preenchimento de vazios BB

In [17]:
# ================================================================
# 2.3) Preenchimento de vazios na Base_BB
# ================================================================

# 1️⃣ Campos textuais – preencher com string vazia ''
campos_texto = [
    'Platform Title', 'Type', 'Deeplink', 'Season Numbers',
    'BB Cast', 'BB Countries', 'BB Directors', 'BB Languages',
    'BB Original Title', 'BB Primary Company', 'BB Primary Country',
    'BB Primary Genre', 'BB Production Companies', 'BB Title', 'IMDb ID'
]

for col in campos_texto:
    if col in Base_BB.columns:
        Base_BB[col] = Base_BB[col].fillna('').astype(str)

# 2️⃣ Campos numéricos – preencher com 0 e converter para inteiro
campos_inteiros = ['Seasons', 'Episodes', 'BB Duration', 'BB Year']

for col in campos_inteiros:
    if col in Base_BB.columns:
        Base_BB[col] = Base_BB[col].fillna(0).astype(int)

# 3️⃣ Campos de identificação – garantir coerência de tipo string
if 'BB UID' in Base_BB.columns:
    Base_BB['BB UID'] = Base_BB['BB UID'].astype(str)

# 4️⃣ Confirmação visual
display(HTML(
    "<h4 style='color:#2a5599;'>✅ Preenchimento e tipagem concluídos com sucesso para <code>Base_BB</code>.</h4>"
))
display(Base_BB.sample(10).style.set_table_styles([
    {'selector': 'th', 'props': [('background-color', '#f0f0f0'), ('text-align', 'center')]},
    {'selector': 'td', 'props': [('padding', '6px 12px')]}
]))


,BB UID,Platform Title,Type,Deeplink,Seasons,Episodes,Season Numbers,BB Cast,BB Countries,BB Directors,BB Duration,BB Languages,BB Original Title,BB Primary Company,BB Primary Country,BB Primary Genre,BB Production Companies,BB Title,BB Year,IMDb ID
11791,106a93fb20ab13573fa686b1d73c8045,INCÊNDIO EM CAMPOS,Movie,http://bcc.cinemateca.org.br/filmes/446913,0,0,,,,,0,,,,,,,INCÊNDIO EM CAMPOS,1968,
7591,1ecec9521d31f176ce3c5c623dd65754,O SOM E O TOQUE,Movie,https://arte1play.com.br/colecoes/arte-na-tecnologia-serie-aXlKYMniyW0/o-som-e-o-toque-YGKB4L0iEP4,0,0,,,,Juliana Barauna,43,,,,,,,O SOM E O TOQUE,2023,
16948,c110adfa4380e08d30d01e7d7036d644,Bang! Bang! The Mafia Gang,Movie,https://www.cultpix.com/movie/bang-bang-the-mafia-gang/1835,0,0,,"Frank Corsentino, Haji, Michael Finn, Marius Mazmanian, Lee McLaughlin, Charles Knapp, Lillian Shrewsbury, Irene Makarewicz, John Bloom, Tyra Tashiro, Luanne Roberts, Uschi Digard, Georgette Dante, Cheri Caffaro, Vincent Barbi, Cara Peters, Michael Stearns, Albert Cole, Lynn Harris, William Kirschner","US, XX",Art Lieberman,70,"und, en",Up Your Alley,Headliner Productions,US,Comedy,"Headliner Productions, Cinépix Film Properties (cfp), Something Weird Video (swv)",Up Your Alley,1971,tt0175284
12713,5a8b5f90d1767fc18061e80ff4c32bda,CASAS DE CÂMBIO - REABERTURA,Movie,http://bcc.cinemateca.org.br/filmes/445859,0,0,,,,,0,,,,,,,CASAS DE CÂMBIO - REABERTURA,1968,
16679,c0e91f80875e7f6d37b6702acc31f9a7,O Vestido,Movie,https://www.clarotvmais.com.br/filme/o-vestido/3107236,0,0,,"Ana Beatriz Nogueira, Leonardo Vieira, Gabriela Duarte, Stela Freitas, Othon Bastos, Ana Lúcia Torre, Daniel Dantas, Paulo José, Renato Borghi, Sura Berditchevsky, Péricles Amin, Paulo André, Adriana Bagno, Alessandra Barros, Ricardo Batista, Daniele Borges, Ricardo Camargos, Augusto Costa, Jackson Costa, Jota Dângelo",BR,Paulo Thiago,117,pt,O Vestido,Vitoria Producoes Cinematograficas,BR,Drama,"Br Petrobrás, Quanta Centro De Produções Cinematográficas, Vitoria Producoes Cinematograficas",O Vestido,2004,tt0413451
22329,75aa9ecebc8e2766e1e9ec63cdf7de2a,Love You Tyler,Movie,https://filmzie.com/content/love-you-tyler-2019,0,0,,"Devon Diffenderfer, Ryan Pater",US,"Ari Itkin, Devon Diffenderfer",9,"english, en",Love You Tyler,,US,Short,,Love You Tyler,2019,tt10238924
17155,8f38e1e719f7c9bab69431e8f6cfc74a,Tarzan and the Treasure,Movie,https://www.cultpix.com/movie/tai-shan-bao-zang/1703,0,0,,"Ke Ba, Gao Ming, Liu Qing, Chin Tu",,Liang Zhefu,83,,Tarzan and the Treasure,,,,,Tarzan and the Treasure,1965,tt15710256
18602,d16dad5047e073ef9d99991511fc0732,História Da Flauta,Movie,https://cultspplay.com.br/video/historia-da-flauta/,0,0,,,,,3,,,,,Music,,História Da Flauta,2022,
35997,3c148e822f74db2ebb5da1281e07d3bc,Trabalhando de Gari,Movie,https://www.vivoplay.com.br/details/movie/trabalhando-de-gari-22566540,0,0,,,,,10,,,,,Children,,Trabalhando de Gari,2014,
3671,4352367bc674560c5008c28d2074dc88,Down On The Corner no estilo de Creedence Clearwater Revival,Movie,https://app.primevideo.com/detail?gti=amzn1.dv.gti.aabba508-8a3b-b67d-538d-fc30618c5341,0,0,,,,Não especificado,2,,,,,,,Down On The Corner no estilo de Creedence Clearwater Revival,1969,


In [18]:
Base_BB.sample(10)

,BB UID,Platform Title,Type,Deeplink,Seasons,Episodes,Season Numbers,BB Cast,BB Countries,BB Directors,BB Duration,BB Languages,BB Original Title,BB Primary Company,BB Primary Country,BB Primary Genre,BB Production Companies,BB Title,BB Year,IMDb ID
22223,c27acdc3ed864302337470b85ad72ad6,Avenger Dogs,Movie,https://filmzie.com/content/avenger-dogs-2019,0,0,,"Joachim Cruise, Luka Pilar, Sam Pitt, Dave Sol...",US,James Snider,71,"english, en",Avenger Dogs,,US,Sci-fi,,Avenger Dogs,2019,tt9724068
24570,47c2b04cba8d9743ba1c4937206a12e6,Em Busca da Casa Perfeita,Series,https://play.max.com/show/15e11f34-5607-43b5-9...,1,1,18,,,,0,,,,,,,,2023,
34654,5163667db17fb7107bca2928d2f2db6f,Novembros,Movie,https://todesplay.com.br/title/novembros/,0,0,,,,Dheik Praia,7,,,,,,,Novembros,2022,
32627,b52a564906df8046a332e6af6c21105a,Peking Opera Blues,Movie,https://watch.plex.tv/movie/peking-opera-blues,0,0,,"Brigitte Lin, Sally Yeh, Cherie Chung Cho-Hung...","HK, CN",Tsui Hark,104,"mandarin, cantonese, zh, en, cn",Peking Opera Blues,Golden Princess Film Production Limited,HK,Action,"Cinema City Film Productions, Film Workshop Co...",Peking Opera Blues,1986,tt0090952
6686,cfa8cbb8f699a038422aee23e5c0cb15,O SAMURAI E O CACIQUE,Movie,https://arte1play.com.br/colecoes/arte1-em-mov...,0,0,,,,Gisele Kato,4,,,,,,,O SAMURAI E O CACIQUE,2023,
21953,a5480ec5e73561474a85ec575eddf772,Houdini & Doyle,Series,https://play.filmboxplus.com/asset/-kEi0T2zyst...,1,10,1,,,,0,,,,,Drama,,Houdini & Doyle,2016,
4045,fb7a69e50bf7b29867c272b3e904042c,Quest 12.5: Xiao’s Report,Movie,https://app.primevideo.com/detail?gti=amzn1.dv...,0,0,,,,Studio Kai,23,,,,,,,Phantasy Star Online 2: Episode Oracle,2022,
10483,3f7b4b73df0869a03a3229577ed35b3b,AERONÁUTICA - MOTORES ROLLS ROYCE B/R-11,Movie,http://bcc.cinemateca.org.br/filmes/448335,0,0,,,,,52,,,,,,,AERONÁUTICA - MOTORES ROLLS ROYCE B/R-11,1968,
323,87b980b6507c6297fb7613c38e9076b4,Conversa de Literatura,Series,https://app.primevideo.com/detail?gti=amzn1.dv...,2,53,"1, 2",,,,0,,,,,,,,2016,
38691,143aa964f75c99404924b41c087ecb73,The Roaring Twenties,Movie,https://www.youtube.com/watch?v=0VVRVfbEF0Y,0,0,,,,,106,,,,,Drama,,The Roaring Twenties,1955,


### 2.3) Criação dos dataframes e verificação dos resultados

In [19]:
# Separando a Base_SAD em filmes e séries
filmesBR_SAD = Base_SAD[Base_SAD["Organização Temporal"] == "NÃO SERIADA"]
seriesBR_SAD = Base_SAD[Base_SAD["Organização Temporal"] != "NÃO SERIADA"]


In [20]:
# Separando a Base_BB em filmes e séries, considerando também a origem BR ou Estrangeira
filmesBR_BB = Base_BB[(Base_BB['BB Countries'].str.contains("BR")) & (Base_BB['Type'].str.contains("Movie"))]
seriesBR_BB = Base_BB[(Base_BB['BB Countries'].str.contains("BR")) & (Base_BB['Type'].str.contains("Series"))]
filmesEstr_BB = Base_BB[(~Base_BB['BB Countries'].str.contains("BR")) & (Base_BB['Type'].str.contains("Movie"))]
seriesEstr_BB = Base_BB[(~Base_BB['BB Countries'].str.contains("BR")) & (Base_BB['Type'].str.contains("Series"))]

In [21]:
# Imprimindo a quantidade de linhas nos datasets originais
print(f"Quantidade de linhas em Base_SAD: {len(Base_SAD)}")
print(f"Quantidade de linhas em Base_BB: {len(Base_BB)}\n")

# Imprimindo a quantidade de linhas em cada dataframe derivado
print(f"Quantidade de linhas em filmesBR_SAD: {len(filmesBR_SAD)}")
print(f"Quantidade de linhas em seriesBR_SAD: {len(seriesBR_SAD)}")
print(f"Quantidade de linhas em filmesBR_BB: {len(filmesBR_BB)}")
print(f"Quantidade de linhas em seriesBR_BB: {len(seriesBR_BB)}")
print(f"Quantidade de linhas em filmesEstr_BB: {len(filmesEstr_BB)}")
print(f"Quantidade de linhas em seriesEstr_BB: {len(seriesEstr_BB)}")

# Comparação entre o input e o output
total_linhas_SAD = len(filmesBR_SAD) + len(seriesBR_SAD)
total_linhas_BB = len(filmesBR_BB) + len(seriesBR_BB) + len(filmesEstr_BB) + len(seriesEstr_BB)

print(f"\nComparação Base_SAD:")
print(f"Total de linhas original: {len(Base_SAD)}")
print(f"Total de linhas após divisão: {total_linhas_SAD}")
print(f"Diferença de linhas Base_SAD: {len(Base_SAD) - total_linhas_SAD}")

print(f"\nComparação Base_BB:")
print(f"Total de linhas original: {len(Base_BB)}")
print(f"Total de linhas após divisão: {total_linhas_BB}")
print(f"Diferença de linhas Base_BB: {len(Base_BB) - total_linhas_BB}")


Quantidade de linhas em Base_SAD: 59911
Quantidade de linhas em Base_BB: 39417

Quantidade de linhas em filmesBR_SAD: 47791
Quantidade de linhas em seriesBR_SAD: 12120
Quantidade de linhas em filmesBR_BB: 816
Quantidade de linhas em seriesBR_BB: 361
Quantidade de linhas em filmesEstr_BB: 33371
Quantidade de linhas em seriesEstr_BB: 4869

Comparação Base_SAD:
Total de linhas original: 59911
Total de linhas após divisão: 59911
Diferença de linhas Base_SAD: 0

Comparação Base_BB:
Total de linhas original: 39417
Total de linhas após divisão: 39417
Diferença de linhas Base_BB: 0


### 2.4) Verificação de títulos homônimos

#### Esta verificação não está criando nem separando nenhum registro ou dataframe e serve 
#### para simples conferência podendo ser desconsiderada a dependeder do contexto

In [22]:
Base_SAD.sample(10)

,Nº CPB,Título Original,Título Alternativo,Classificação,Gênero,Organização Temporal,Duração Obra,Nº de Episódios,Ano de Produção,Produtora,Diretor,País de Origem
20503,B1402167900000,"QUERO FAZER PORNÔ, MAS NÃO POSSO 2",NaN,COMUM,FICÇÃO,NÃO SERIADA,76.00,NaN,2014,WS PRODUÇÕES CINEMATOGRÁFICAS E ESTÚDIOS LTDA,EDUARDO ADILSON MASTROGIOVANNI,BR
6603,B0700742600000,CRIADORES DO BRASIL,NaN,BRASILEIRA INDEPENDENTE CONSTITUINTE DE ESPAÇO...,DOCUMENTÁRIO,NÃO SERIADA,52.00,NaN,2005,ESTAÇÃO ELÉTRICA PRODUÇÃO DE CINEMA E VÍDEO LTDA.,RENE TRINDADE GOYA FILHO,BR
57827,B2500116900000,APANHADOR DE ALMAS,NaN,BRASILEIRA INDEPENDENTE CONSTITUINTE DE ESPAÇO...,FICÇÃO,NÃO SERIADA,99.00,NaN,2025,IRON CHEST PRODUCTIONS LTDA,"NELSON BOTTER JUNIOR, FERNANDO ALONSO MOYSES",BR
15188,B1201609300000,ANTES QUE O VERÃO ACABE,NaN,BRASILEIRA INDEPENDENTE CONSTITUINTE DE ESPAÇO...,FICÇÃO,NÃO SERIADA,16.03,NaN,2012,MARILIA NOGUEIRA SILVEIRA,MARILIA NOGUEIRA SILVEIRA,BR
23242,B1500156100000,VISITA AO ESCRITORIO DE FERMIN VAZQUEZ,NaN,BRASILEIRA INDEPENDENTE CONSTITUINTE DE ESPAÇO...,DOCUMENTÁRIO,NÃO SERIADA,12.57,NaN,2014,IRMÃOS GUERRA FILMES LTDA,HELENA ROMANO GUERRA,BR
11984,B1001285800000,002 SURF,NaN,NÃO INFORMADO,NÃO CLASSIFICADA,NÃO SERIADA,59.95,NaN,1990,PEDRO CEZAR DUARTE GUIMARÃES,PEDRO CEZAR DUARTE GUIMARÃES,BR
51212,B2300311400000,PRIMEIROS POVOS,NaN,BRASILEIRA INDEPENDENTE CONSTITUINTE DE ESPAÇO...,DOCUMENTÁRIO,SERIADA EM TEMPORADA ÚNICA,263.92,5.0,2023 A 2023,BONNE PIOCHE TELEVISION | CANAL AZUL PRODUÇÕES...,"MIKE MAGIDSON, ALEXANDRE JEAN JACQUES ALAIN BO...","FR, BR"
58792,B2500246500000,MUITO ALÉM DA TEMPESTADE,NaN,BRASILEIRA CONSTITUINTE DE ESPAÇO QUALIFICADO,DOCUMENTÁRIO,NÃO SERIADA,32.77,NaN,2025,ELIANA CAMEJO ASSESSORIA DE COMUNICACAO E MARK...,VITOR SILVA VILELLA,BR
59843,B2500388200000,MAINSTREAM,NaN,BRASILEIRA INDEPENDENTE CONSTITUINTE DE ESPAÇO...,VÍDEOMUSICAL,NÃO SERIADA,3.55,NaN,2025,DANIELE SALES SOUSA LOPES,WELLINGTON BARBOSA DOS SANTOS,BR
41448,B2000340900000,GARIMPANDO COM MARCELO SAMPAIO TEMPORADA 3,NaN,BRASILEIRA INDEPENDENTE CONSTITUINTE DE ESPAÇO...,VARIEDADES,SERIADA EM MÚLTIPLAS TEMPORADAS,720.00,24.0,2020 A 2020,MARCELO EDUARDO SAMPAIO,MARCELO EDUARDO SAMPAIO,BR


In [23]:
# ================================================================
# 3.1) Verificação de títulos homônimos nas bases SAD e BB
# ================================================================

def verificar_homonimos(df, nome_df, col_titulo, col_ano, col_diretor, col_id, col_titulo_alt=None):
    """
    Identifica títulos homônimos dentro de uma base,
    considerando título (e opcionalmente título alternativo), ano, diretor e identificadores únicos.
    """
    # Define as colunas de agrupamento dinamicamente
    colunas_grp = [col_titulo, col_ano, col_diretor]
    if col_titulo_alt and col_titulo_alt in df.columns and col_titulo_alt != col_titulo:
        colunas_grp.insert(1, col_titulo_alt)  # insere o alternativo logo após o título principal

    # Agrupamento e contagem de identificadores distintos
    repeticoes = (
        df.groupby(colunas_grp)[col_id]
        .nunique()
        .reset_index(name='Qtd_IDs')
    )

    # Filtra apenas casos com mais de um identificador
    repeticoes = repeticoes[repeticoes['Qtd_IDs'] > 1]

    # Exibição formatada
    display(HTML(f"<h4 style='color:#2a5599;'>🎬 Verificação de homônimos — {nome_df}</h4>"))
    if repeticoes.empty:
        display(HTML("<p style='color:green;'>✅ Nenhum título homônimo encontrado.</p><hr>"))
    else:
        display(repeticoes.style.set_table_styles([
            {'selector': 'th', 'props': [('background-color', '#f0f0f0'), ('text-align', 'center')]},
            {'selector': 'td', 'props': [('padding', '6px 12px')]}
        ]))
        print(f"\nQuantidade de títulos homônimos em {nome_df}: {len(repeticoes)}\n")
        display(HTML("<hr style='margin:25px 0;'>"))


# ================================================================
# Aplicação às bases
# ================================================================

# --- Filmes e séries brasileiras (SAD)
verificar_homonimos(
    filmesBR_SAD,
    "Filmes Brasileiros (SAD)",
    "Título Original",
    "Ano de Produção",
    "Diretor",
    "Nº CPB"
)

verificar_homonimos(
    seriesBR_SAD,
    "Séries Brasileiras (SAD)",
    "Título Original",
    "Ano de Produção",
    "Diretor",
    "Nº CPB"
)

# --- Filmes e séries brasileiras (BB)
verificar_homonimos(
    filmesBR_BB,
    "Filmes Brasileiros (BB)",
    "Platform Title",
    "BB Year",
    "BB Directors",
    "BB UID",
    col_titulo_alt="BB Original Title"
)

verificar_homonimos(
    seriesBR_BB,
    "Séries Brasileiras (BB)",
    "Platform Title",
    "BB Year",
    "BB Directors",
    "BB UID",
    col_titulo_alt="BB Original Title"
)

# --- Filmes e séries estrangeiras (BB)
verificar_homonimos(
    filmesEstr_BB,
    "Filmes Estrangeiros (BB)",
    "Platform Title",
    "BB Year",
    "BB Directors",
    "BB UID",
    col_titulo_alt="BB Original Title"
)

verificar_homonimos(
    seriesEstr_BB,
    "Séries Estrangeiras (BB)",
    "Platform Title",
    "BB Year",
    "BB Directors",
    "BB UID",
    col_titulo_alt="BB Original Title"
)


,Título Original,Ano de Produção,Diretor,Qtd_IDs
939,A CIDADE,2012,LILIANA SULZBACH,2
2766,ABSURDO - ALMA LIVRE FEAT. CELSO MORETTI,2023,ROBERTO MORAES RIBEIRO DE MENDONCA,2
3068,AGENDA,2020,RODRIGO CASTRO FORTE CARDOSO,2
3106,AGORA VAI,2023,"MATHEUS MALAFAIA TEIXEIRA, DANIELA GLEISER",2
3298,ALEGRIA,2006,SERGIO SILVA GOMES,2
3368,ALGODÃO DOCE,2025,"JADSON PLANZO BARBOSA, FILIPE OLIVEIRA SOUZA",2
4429,ANJOS EM AÇÃO 2021 - A ESPERANÇA DE UM NOVO TEMPO,2021,ANTONIO OLIVEIRA TOSTES,2
4998,ARNALDO ANTUNES,2013,BRUNO LEVINSON,2
6188,BALA E FOGO,2024,RAFAEL MACIEL SALGADO CARMO,2
7525,BRASIL DAS GERAIS,2020,MARINA SUELI CUNHA MENDES,2



Quantidade de títulos homônimos em Filmes Brasileiros (SAD): 111



,Título Original,Ano de Produção,Diretor,Qtd_IDs
165,52 PIZZAS,2011 A 2011,JOSE EDUARDO NICOLAU,2
409,A MULHER INVISÍVEL,2011 A 2011,MIGUEL ARRAES DE ALENCAR FILHO,2
684,AGROCULTURA,2021 A 2021,CEZAR AUGUSTO FREIRE DA SILVA,2
1013,AO PONTO - INFORMAÇÃO DO JEITO QUE VOCÊ GOSTA,2005 A 2005,SÉRGIO MORAES CASTANHEIRA BRANDÃO,2
1126,ARTE CONTEMPORÂNEA - 2A. TEMPORADA,2017 A 2017,FABRICIO RAYMUNDO TIMM,2
2145,CARNAVAL SEXPRIVE,2020 A 2020,FABIO COSTA MORAES,2
2510,CINEMATÓGRAFO,2018 A 2018,JOHANES,2
2593,CNN VIAGEM & GASTRONOMIA,2022 A 2022,BRUNO TEIXEIRA CHIARIONI,2
2676,COM FOME DE VIAGEM,2023 A 2023,ALEXANDRO DE ALMEIDA FERREIRA,2
2827,CONHECENDO O BRASIL,2017 A 2017,DEBORAH CAROLINE CEARAMICOLI BARBOSA SILVESTRE,2



Quantidade de títulos homônimos em Séries Brasileiras (SAD): 68



,Platform Title,BB Original Title,BB Year,BB Directors,Qtd_IDs
539,O clã das jiboias: o Jiu-Jitsu da Amazônia para o mundo,O Clã das Jiboias: O Jiu-Jitsu da Amazônia para o Mundo,2023,Heraldo Daniel Moraes,2



Quantidade de títulos homônimos em Filmes Brasileiros (BB): 1



,Platform Title,BB Original Title,BB Year,BB Directors,Qtd_IDs
3779,Assassinos,,1997,Mathieu Kassovitz,2
7789,Consequências,,2023,,2
8782,DOPS: Uma Arqueologia da Violência,,2024,Sanzio Canfora,2
10200,"ESG - Quem se importa, ganha",,2023,"Luciano Oreggia,Pedro Saad",2
11023,Erupção de Tubarões,,2024,,2
11085,Especial Roberto Carlos 2014,,2014,Luiz Gleiser,2
12997,Gilberto Gil: A God In His Garden,,2020,Angel Diez,2
13888,Hitler vs. Picasso - A Obsessão Nazista pela Arte,,2018,Claudio Poli,2
15671,Jessie J Plays Baloise Sessions,,2023,Roli Bärlocher,2
16154,Kids' Choice Awards 2025,,2025,,2



Quantidade de títulos homônimos em Filmes Estrangeiros (BB): 27



,Platform Title,BB Original Title,BB Year,BB Directors,Qtd_IDs
172,A Reforma de Lindsey,Inn the Works,2021,,2
686,Botched: Pesadelo na Bariátrica,Botched Bariatrics,2024,,2
1571,Em Busca da Casa Perfeita,,2021,,2
1994,Harry Potter: Bruxos da Confeitaria,Harry Potter: Wizards of Baking,2024,,2
2185,Invasão a Paris,,2024,Hans Herbots,2
2212,JA 1ª,,0,,2
2686,Meet the Queens of Drag Race Philippines: Slaysian Royale,,2025,,2
3039,O Caso Sherri Papini,,2025,,2
3573,Publio: O Sequestro Sem Fim,Publio. El secuestro sin fin,2024,Fèlix Colomer Vallès,2
3793,Rubiales vs Hermoso: O Beijo da Copa do Mundo,,2025,,2



Quantidade de títulos homônimos em Séries Estrangeiras (BB): 10



## 3) Processamento para uniformização de campos de texto e numéricos

### 3.1) Definição das funções de processamento

In [24]:
# Criação de cópias limpas dos DataFrames
filmesBR_SAD_limpo = filmesBR_SAD.copy()
seriesBR_SAD_limpo = seriesBR_SAD.copy()

filmesBR_BB_limpo = filmesBR_BB.copy()
seriesBR_BB_limpo = seriesBR_BB.copy()

filmesEstr_BB_limpo = filmesEstr_BB.copy()
seriesEstr_BB_limpo = seriesEstr_BB.copy()


In [25]:
# Função para transformar numerais romanos para arábicos
def roman_to_arabic(roman):
    roman_numerals = {
        'i': 1,
        'ii': 2,
        'iii': 3,
        'iv': 4,
        'v': 5,
        'vi': 6,
        'vii': 7,
        'viii': 8,
        'ix': 9,
        'x': 10
    }
    
    return roman_numerals.get(roman.lower(), None)

In [26]:
def _colapsar_sigla_pontilhada(s):
    if not isinstance(s, str):
        return s
    # casa sequências de letra+ponto repetidas (2+ letras), opcional letra final e ponto
    # exemplos cobertos: "L.A.P.A.", "U.S.A", "F.B.I.", "L. A. P. A."
    return re.sub(r'(?i)(?:\b[a-z]\s*\.){2,}[a-z]?\s*\.?', 
                  lambda m: re.sub(r'[\s\.]', '', m.group(0)), 
                  s)

In [27]:
def nonempty_str(series_or_str):
    s = pd.Series(series_or_str) if not isinstance(series_or_str, pd.Series) else series_or_str
    s = s.astype(str).str.strip()
    return s.notna() & s.ne('') & s.str.lower().ne('nan') & s.str.lower().ne('none')

In [28]:
# Função para realizar as transformações de strings
def process_string3(s):
    if not isinstance(s, str):  # Se não for uma string, retorne como está
        return s
    
    # Corrigir espaçamento antes e depois de vírgulas
    s = re.sub(r'\s*,\s*', ', ', s)
    s = s.lower().strip()  # Converter texto para minúsculo e remover espaços no começo/fim
    
    # Remover caracteres não latinos (Chineses por exemplo)
    s = re.sub(r'[^\u0000-\u024F\u1E00-\u1EFF]+', '', s)
    
    # Remover referências de temporadas
    patterns = [
    r'(\s*-?\s*season\s*\d+)$',
    r'(\s*-?\s*\d+\s*[ºª°]?\s*temporada)$',
    r'(\s*-?\s*\d+[aª]?\s*temp)$',
    r'(\s*-?\s*\d+rd\s*season)$',
    r'(\s*-?\s*\d+th\s*season)$',
    r'(\s*-?\s*(primeira|segunda|terceira|quarta|quinta|sexta|sétima|oitava|nona|décima)\s*temporada)$',
    ]
    for pattern in patterns:
        s = re.sub(pattern, '', s, flags=re.IGNORECASE)
    
    
    # Converter numerais romanos para arábicos
    words = s.split()
    for i, word in enumerate(words):
        if word in ['i', 'ii', 'iii', 'iv', 'v', 'vi', 'vii', 'viii', 'ix', 'x']:
            words[i] = str(roman_to_arabic(word))
    s = ' '.join(words)
    
    # Substituir ° por o antes de remover acentos
    s = s.replace('°', 'o')
    s = s.replace('°', 'o')
    s = s.replace('ª', 'a')

    # COLAPSAR SIGLAS PONTILHADAS ANTES DE REMOVER PONTUAÇÃO
    s = _colapsar_sigla_pontilhada(s)
    
    # Transformar caracteres de pontuação em espaços
    for char in string.punctuation:
        s = s.replace(char, ' ')
    
    # Converter caracteres acentuados para sua versão sem acentuação
    s = unidecode.unidecode(s)
    
    # Remover palavras comuns
    common_words = ['o', 'a', 'os', 'as', 'um', 'uma', 'do', 'da', 'dos', 'das', 'e', 'the']
    s = ' '.join([word for word in s.split() if word not in common_words])
    
    # Remover espaços excessivos
    s = re.sub(' +', ' ', s)   
    

    
    return s




In [29]:
def remove_palavras_específicas2(s):
    """
    Remove múltiplas siglas empresariais no final do nome da produtora.
    Exemplos:
      - 'LTDA - ME'
      - 'EIRELI - ME'
      - 'LTDA. - ME - BAIXADO'
    """
    if not isinstance(s, str):
        return s

    s = " " + s.strip() + " "

    # Padroniza pontuação comum
    s = re.sub(r'[\.\-–]+', ' ', s)

    # Lista ampliada de siglas empresariais comuns
    palavras_especificas = [
        's', 'me', 'ltd', 'ltda', 'inc', 'sa', 'eireli', 'mei', 'corp', 'epp', 'ei'
    ]

    # Remove todas as ocorrências dessas siglas no final (pode haver mais de uma)
    while True:
        s_antigo = s
        for palavra in palavras_especificas:
            s = re.sub(rf'\s{palavra}\s*$', ' ', s, flags=re.IGNORECASE)
        if s == s_antigo:
            break

    return s.strip()

In [30]:
#  Remover CPFs ou blocos numéricos no início

def remove_cpf_inicio(s):
    """
    Remove CPF ou blocos numéricos no início da string.
    Casos tratados:
      - '123456789 Nome'
      - '123.456.789 Nome'
    """
    if not isinstance(s, str):
        return s

    s = s.strip()

    # Caso 1: primeiros 9 caracteres são todos dígitos + espaço
    if re.match(r'^\d{9}\s', s):
        return s[10:].strip()

    # Caso 2: padrão CPF formatado com pontos — 11 dígitos no formato XXX.XXX.XXX
    s = re.sub(r'^\d{3}\.\d{3}\.\d{3}\s+', '', s)

    return s.strip()


In [31]:
#  Remover CPFs ou blocos numéricos no final

def remove_cpf_fim(s):
    """
    Remove CPF no FINAL do texto da produtora.
    Casos tratados:
      - '... 123456789'
      - '... 123.456.789'
      - '... - 123456789' / '... - 123.456.789'
      - '... (123456789)' / '... (123.456.789)'
    Não altera outros números, pois exige exatamente 9 dígitos (com ou sem pontos).
    """
    if not isinstance(s, str):
        return s

    s = s.strip()

    # remove bloco final: [opcional ' - ' ou parênteses] + (XXX.XXX.XXX | XXXXXXXXX) + final de string
    s = re.sub(r'[\s\-\–\(\)]*(?:\d{3}\.\d{3}\.\d{3}|\d{9})\s*$', '', s, flags=re.IGNORECASE)

    return s.strip()


In [32]:
# Função para transformar o campo de texto do ano de produção em dois campos numericos: Ano inicial e Ano final
def extract_ano_inicial(year_value):
    # Converter o valor para string
    year_string = str(year_value)
    
    # Se 'A' estiver presente, extraia o ano antes do 'A'
    if 'A' in year_string:
        return int(year_string.split(' A ')[0])
    # Se for apenas um ano, retorne esse ano
    elif year_string.isdigit():
        return int(year_string)
    else:
        return None

def extract_ano_final(year_value):
    # Converter o valor para string
    year_string = str(year_value)
    
    # Se 'A' estiver presente, extraia o ano após o 'A'
    if 'A' in year_string:
        return int(year_string.split(' A ')[1])
    # Se for apenas um ano, retorne esse mesmo ano
    elif year_string.isdigit():
        return int(year_string)
    else:
        return None

In [33]:
def remove_last_baixada(s):
    """
    Remove ocorrências de 'baixado' ou 'baixada' (com hífen, espaço ou parênteses).
    Exemplos tratados:
      - '(BAIXADA)'
      - '- BAIXADO'
      - '- BAIXADA'
      - '(BAIXADO)'
    """
    if not isinstance(s, str):
        return s

    s = s.strip()

    # Remove qualquer ocorrência no final, com ou sem parênteses ou hífen
    pattern = r'[\s\-\(]*(baixad[ao])[\s\)]*$'
    return re.sub(pattern, '', s, flags=re.IGNORECASE).strip()


In [34]:
particles_pt = {'da','de','do','das','dos'}

def _normalize_name_basic(s: str) -> str:
    s = '' if pd.isna(s) else str(s).strip().lower()
    s = unidecode.unidecode(s)
    s = re.sub(r'[^\w\s]', ' ', s)
    s = re.sub(r'\s+', ' ', s).strip()
    return s

def surnames_from_raw(raw: str) -> set:
    """
    Divide por VÍRGULA, normaliza cada nome e extrai o sobrenome 'de verdade':
    - último token; se penúltimo ∈ {da,de,do,das,dos}, pega 'penúltimo + último' (ex.: 'da silva').
    """
    if pd.isna(raw) or not str(raw).strip():
        return set()
    out = set()
    for p in str(raw).split(','):
        n = _normalize_name_basic(p)
        if not n:
            continue
        toks = n.split()
        if not toks:
            continue
        # '... da silva' -> 'da silva'
        if len(toks) >= 2 and toks[-2] in particles_pt:
            out.add(toks[-2] + ' ' + toks[-1])
        else:
            out.add(toks[-1])
    return out

def tokens_empresas(s: str) -> set:
    """
    Usa campos *_processado (já sem ltda, me, etc.) e quebra em tokens distintivos.
    """
    if pd.isna(s) or not str(s).strip():
        return set()
    t = str(s).strip().split()
    return set([w for w in t if len(w) > 2])  # ignora tokens 1–2 letras

### 3.2) Aplicação das Funções para uniformização de campos textuais

In [35]:
# ===================================================================================
# 3.2 Aplicação das Funções para uniformização de campos textuais
# ===================================================================================

# ================================================
# Bases SAD
# ================================================
for df in [filmesBR_SAD_limpo, seriesBR_SAD_limpo]:
    
    # --- Normalização base (acentos, pontuação, caixa etc.)
    df['Título Original_processado'] = df['Título Original'].apply(process_string3)
    df['Diretor_processado']         = df['Diretor'].apply(process_string3)
    df['Produtora_processada']       = df['Produtora'].apply(process_string3)
    
    # --- Remoção de CPF no início e no final das strings
    df['Produtora_processada'] = df['Produtora_processada'].apply(remove_cpf_inicio)
    df['Produtora_processada'] = df['Produtora_processada'].apply(remove_cpf_fim)
    
    # --- Remoção de indicações de "baixado(a)" em diferentes formatos
    df['Produtora_processada'] = df['Produtora_processada'].apply(remove_last_baixada)
    df['Diretor_processado']   = df['Diretor_processado'].apply(remove_last_baixada)
    
    # --- Remoção de siglas empresariais (LTDA, ME, EIRELI etc.)
    df['Produtora_processada'] = df['Produtora_processada'].apply(remove_palavras_específicas2)


# ================================================
# Bases BB_BR
# ================================================
for df in [filmesBR_BB_limpo, seriesBR_BB_limpo]:
    
    # --- Normalização base
    df['Platform Title_processado']          = df['Platform Title'].apply(process_string3)
    df['BB Original Title_processado']       = df['BB Original Title'].apply(process_string3)
    df['BB Primary Company_processado']      = df['BB Primary Company'].apply(process_string3)
    df['BB Production Companies_processado'] = df['BB Production Companies'].apply(process_string3)
    df['BB Directors_processado']            = df['BB Directors'].apply(process_string3)
    df['BB Title_processado']                = df['BB Title'].apply(process_string3)
    
    # --- Remoção de CPF no início e no final das strings (para produtoras)
    df['BB Production Companies_processado'] = df['BB Production Companies_processado'].apply(remove_cpf_inicio)
    df['BB Production Companies_processado'] = df['BB Production Companies_processado'].apply(remove_cpf_fim)
    df['BB Primary Company_processado']      = df['BB Primary Company_processado'].apply(remove_cpf_inicio)
    df['BB Primary Company_processado']      = df['BB Primary Company_processado'].apply(remove_cpf_fim)
    
    # --- Remoção de "baixado(a)" e variantes
    df['BB Production Companies_processado'] = df['BB Production Companies_processado'].apply(remove_last_baixada)
    df['BB Primary Company_processado']      = df['BB Primary Company_processado'].apply(remove_last_baixada)
    
    # --- Remoção de siglas empresariais (LTDA, ME, EIRELI etc.)
    df['BB Production Companies_processado'] = df['BB Production Companies_processado'].apply(remove_palavras_específicas2)
    df['BB Primary Company_processado']      = df['BB Primary Company_processado'].apply(remove_palavras_específicas2)


# ================================================
# Bases BB_estrangeiras
# ================================================

for df in [filmesEstr_BB_limpo, seriesEstr_BB_limpo]:
    # 1) Normalização base
    df['Platform Title_processado']          = df['Platform Title'].apply(process_string3)
    df['BB Original Title_processado']       = df['BB Original Title'].apply(process_string3)
    df['BB Primary Company_processado']      = df['BB Primary Company'].apply(process_string3)
    df['BB Production Companies_processado'] = df['BB Production Companies'].apply(process_string3)
    df['BB Directors_processado']            = df['BB Directors'].apply(process_string3)
    df['BB Title_processado']                = df['BB Title'].apply(process_string3)

    # 2) Remoção de CPF no início e no fim (produtoras)
    df['BB Production Companies_processado'] = df['BB Production Companies_processado'].apply(remove_cpf_inicio).apply(remove_cpf_fim)
    df['BB Primary Company_processado']      = df['BB Primary Company_processado'].apply(remove_cpf_inicio).apply(remove_cpf_fim)

    # 3) Remoção de “baixado/baixada”
    df['BB Production Companies_processado'] = df['BB Production Companies_processado'].apply(remove_last_baixada)
    df['BB Primary Company_processado']      = df['BB Primary Company_processado'].apply(remove_last_baixada)

    # 4) Remoção de siglas empresariais
    df['BB Production Companies_processado'] = df['BB Production Companies_processado'].apply(remove_palavras_específicas2)
    df['BB Primary Company_processado']      = df['BB Primary Company_processado'].apply(remove_palavras_específicas2)



In [36]:
# ===================================================================================
# 3.3 Amostras das bases processadas
# ===================================================================================

def mostrar_amostra(df, titulo):
    display(HTML(f"<h4 style='color:#006699;'>{titulo}</h4>"))
    display(df.sample(5).style.set_table_attributes("style='display:inline'").set_caption(titulo))
    display(HTML("<hr style='margin:25px 0;'>"))

# Exibir amostras
mostrar_amostra(filmesBR_SAD_limpo, "🎬 Filmes Brasileiros (SAD)")
mostrar_amostra(seriesBR_SAD_limpo, "📺 Séries Brasileiras (SAD)")
mostrar_amostra(filmesBR_BB_limpo, "🎞️ Filmes Brasileiros (BB)")
mostrar_amostra(seriesBR_BB_limpo, "📡 Séries Brasileiras (BB)")
mostrar_amostra(filmesEstr_BB_limpo, "🌍 Filmes Estrangeiros (BB)")
mostrar_amostra(seriesEstr_BB_limpo, "🌍 Séries Estrangeiras (BB)")



,Nº CPB,Título Original,Título Alternativo,Classificação,Gênero,Organização Temporal,Duração Obra,Nº de Episódios,Ano de Produção,Produtora,Diretor,País de Origem,Título Original_processado,Diretor_processado,Produtora_processada
23436,B1500199000000,BOOTSY COLLINS,nan,BRASILEIRA CONSTITUINTE DE ESPAÇO QUALIFICADO,VÍDEOMUSICAL,NÃO SERIADA,56.230000,nan,2013,SERVICO SOCIAL DO COMERCIO - SESC - ADMINISTRACAO REGIONAL NO ESTADO DE SAO PAULO,CAMILA TODARO SANTOS DE MIRANDA,BR,bootsy collins,camila todaro santos de miranda,servico social comercio sesc administracao regional no estado de sao paulo
54571,B2400215900000,ESCADA,nan,BRASILEIRA INDEPENDENTE CONSTITUINTE DE ESPAÇO QUALIFICADO,VÍDEOMUSICAL,NÃO SERIADA,3.270000,nan,2024,DANIEL BAROSA PRODUCOES CINEMATOGRAFICAS,GUILHERME CARNEIRO HOMEM ALVES,BR,escada,guilherme carneiro homem alves,daniel barosa producoes cinematograficas
13285,B1101417800000,COICES E COCEIRAS,nan,BRASILEIRA CONSTITUINTE DE ESPAÇO QUALIFICADO,ANIMAÇÃO,NÃO SERIADA,3.600000,nan,2010,EDUARDO ROBERTO DE SOUZA,EDUARDO ROBERTO DE SOUZA,BR,coices coceiras,eduardo roberto de souza,eduardo roberto de souza
27915,B1600428300000,SERGE EREGE - RHYTHMN OF THE DAY,nan,BRASILEIRA INDEPENDENTE CONSTITUINTE DE ESPAÇO QUALIFICADO,VÍDEOMUSICAL,NÃO SERIADA,4.920000,nan,2015,IDCLIP FILMES EIRELI,RUDA CABRAL DE MEDEIROS BARROS,BR,serge erege rhythmn of day,ruda cabral de medeiros barros,idclip filmes
7252,B0700809000000,SAIA SANTA,nan,NÃO INFORMADO,NÃO CLASSIFICADA,NÃO SERIADA,14.000000,nan,2004,MAURO D ADDIO DA SILVA,MAURO D ADDIO DA SILVA,BR,saia santa,mauro d addio silva,mauro d addio silva


,Nº CPB,Título Original,Título Alternativo,Classificação,Gênero,Organização Temporal,Duração Obra,Nº de Episódios,Ano de Produção,Produtora,Diretor,País de Origem,Título Original_processado,Diretor_processado,Produtora_processada
46555,B2200084500000,TRANSLÚCIDOS,nan,COMUM,JORNALÍSTICA,SERIADA DE DURAÇÃO INDETERMINADA,336.000000,14.000000,2022 A 2022,A COR DA VOZ CRIAÇÕES E PRODUÇÕES LTDA. | ANA LUCIA DE CARVALHO RIBEIRO,ANA LUCIA DE CARVALHO RIBEIRO,BR,translucidos,ana lucia de carvalho ribeiro,cor voz criacoes producoes ltda ana lucia de carvalho ribeiro
47187,B2200184300000,BELEZA A PROVA,nan,BRASILEIRA CONSTITUINTE DE ESPAÇO QUALIFICADO,REALITY-SHOW,SERIADA EM MÚLTIPLAS TEMPORADAS,720.000000,12.000000,2022 A 2022,AGROMIX TELEVISAO LTDA,VALDECIR SHIROMA DE PAULA,BR,beleza prova,valdecir shiroma de paula,agromix televisao
26796,B1600138700000,BRASIL VISTO DE CIMA - 3ª TEMPORADA,nan,BRASILEIRA INDEPENDENTE CONSTITUINTE DE ESPAÇO QUALIFICADO,DOCUMENTÁRIO,SERIADA EM MÚLTIPLAS TEMPORADAS,1050.000000,35.000000,2016 A 2016,GLOBOSAT PROGRAMADORA LTDA (BAIXADA) | HORIZONTE CONTEUDOS LTDA (BAIXADA) | MARIA TV COMUNICAÇÃO LTDA,ANDRE BASEGGIO,BR,brasil visto de cima,andre baseggio,globosat programadora ltda baixada horizonte conteudos ltda baixada maria tv comunicacao
20867,B1402204500000,HOJE É DIA DE MÚSICA,nan,BRASILEIRA INDEPENDENTE CONSTITUINTE DE ESPAÇO QUALIFICADO,DOCUMENTÁRIO,SERIADA,560.000000,10.000000,2014 A 2014,BRASIL DISTRIBUTION LLC | CONSPIRAÇÃO FILMES ENTRETENIMENTO 3º MILÊNIO LTDA.,HUGO SERGIO KOATZ SUKMAN,"US, BR",hoje dia de musica,hugo sergio koatz sukman,brasil distribution llc conspiracao filmes entretenimento 3o milenio
15991,B1201690600000,BROTHERS NA GRINGA,nan,BRASILEIRA INDEPENDENTE CONSTITUINTE DE ESPAÇO QUALIFICADO,VARIEDADES,SERIADA EM TEMPORADA ÚNICA,240.000000,12.000000,2012 A 2012,RIMA PRODUÇÕES LTDA,FLAVIA DO NASCIMENTO TACONI,BR,brothers na gringa,flavia nascimento taconi,rima producoes


,BB UID,Platform Title,Type,Deeplink,Seasons,Episodes,Season Numbers,BB Cast,BB Countries,BB Directors,BB Duration,BB Languages,BB Original Title,BB Primary Company,BB Primary Country,BB Primary Genre,BB Production Companies,BB Title,BB Year,IMDb ID,Platform Title_processado,BB Original Title_processado,BB Primary Company_processado,BB Production Companies_processado,BB Directors_processado,BB Title_processado
34785,3e5b6e66599466d5ab81a7cbfbd48d57,Sample,Movie,https://todesplay.com.br/title/sample/,0,0,,,BR,Ana Julia Travia,0,portuguese,Sample,,BR,Short,,Sample,2018,tt13659220,sample,sample,,,ana julia travia,sample
34277,090ee2b3862f76125205b817335c424b,Coletânea De Histórias Extremamente Curtas,Movie,https://tamandua.tv.br/filme/?name=coletanea_de_historias_extremamente_curtas,0,0,,"César Baccan, Maira Campos, Marcelo Colaiácovo, Henrique Colela, Alexandre Couso, Denise Fraga, Eike Gabriel, Geanluka Guedes, Carlos Ladessa, Henrique Mattera, Argemiro Meirelles, Valentina Meirelles, Miguel Monori, Fabio Nassar, Luiz Ramalho, Pedro Ribeiro, Donny Rodrigues, José Romero",BR,Pedro Fraga Villaça,11,"portuguese, pt, english",Collection of Extremely Short Stories,Café Royal,BR,Comedy,Café Royal,Collection of Extremely Short Stories,2022,tt22301910,coletanea de historias extremamente curtas,collection of extremely short stories,cafe royal,cafe royal,pedro fraga villaca,collection of extremely short stories
20887,f7f7b0fae6da5745756e33f20dbb3866,Incêndio,Movie,https://embaubaplay.com/catalogo/incendio/,0,0,,"Joana Craveiro, António Moniz Pereira, Joana Gama, Luis Duque, António Seabra","PT, XX, BR","Miguel Seabra Lopes, Karen Akerman",23,"portuguese, pt, und",Fire,Terratreme,PT,Comedy,Terratreme,Fire,2011,tt2234147,incendio,fire,terratreme,terratreme,miguel seabra lopes karen akerman,fire
34319,b99d8d5f3245c799d84530692a01e58c,O Som do Silêncio,Movie,https://tamandua.tv.br/filme/?name=o_som_do_silencio,0,0,,"Camila Bonifácio, Zitta Carmo, Nelia Carvalho, Gean Pedro, Marcus Vinicius",BR,David Aynan,18,portuguese,O Som do Silêncio,Palenque Filmes,BR,Short,Palenque Filmes,O Som do Silêncio,2017,tt29925783,som silencio,som silencio,palenque filmes,palenque filmes,david aynan,som silencio
29732,8e17586b0b2f5fd79a91973d33359af0,A Dona da História,Movie,https://www.netflix.com/br/title/80082016,0,0,,"Marieta Severo, Débora Falabella, Antônio Fagundes, Rodrigo Santoro, Giulia Gam, Marcos Oliveira, Rodrigo Penna, Fernanda Lima, Renata Sorrah, Gilberto Hernández, Daniel de Oliveira, José Carlos Pieri, Antonio Fragoso, Ana Furtado, Bianca Byington, Dedina Bernardelli, Charles Paraventi, Paulo Barboza, Clovis Bueno, Jô Soares","US, BR",Daniel Filho,87,"pt, en",Owner of the Story,"Buena Vista International, Inc.",BR,Drama,"Globo Filmes, Miravista, Lereby Produções, Gativideo, Buena Vista International, Inc.",Owner of the Story,2004,tt0395994,dona historia,owner of story,buena vista international,globo filmes miravista lereby producoes gativideo buena vista international,daniel filho,owner of story


,BB UID,Platform Title,Type,Deeplink,Seasons,Episodes,Season Numbers,BB Cast,BB Countries,BB Directors,BB Duration,BB Languages,BB Original Title,BB Primary Company,BB Primary Country,BB Primary Genre,BB Production Companies,BB Title,BB Year,IMDb ID,Platform Title_processado,BB Original Title_processado,BB Primary Company_processado,BB Production Companies_processado,BB Directors_processado,BB Title_processado
2071,3ce9865638d72d20b4d6065123aae693,Little Baby Bum Classic - Cantigas para Crianças,Series,https://app.primevideo.com/detail?gti=amzn1.dv.gti.499675ad-e30b-4ce4-b896-de4a79ed8baf,7,22,"1, 2, 3, 4, 5, 6, 7",,BR,Cannis Holder,0,,Little Baby Bum Classic,,BR,Animation,,Little Baby Bum Classic,2018,,little baby bum classic cantigas para criancas,little baby bum classic,,,cannis holder,little baby bum classic
33376,8ec443f37bf9da2a6fdc2e0ed5baa7d7,Máquina da Fama,Series,https://mais.sbt.com.br/program/6358207093112,1,39,4,"Paula Bressann, Cássia Raquel, Fabio Rabello, Nathan Barone, Paulo Castagnoli, Caíque Gama, Patricia Abravanel, Maisa, Adriane Galisteu, Tati Zaqui, Nicholas Torres, Luciana Gimenez, Kelly Key, Danilo Gentili, Nicole Bahls, Wanessa Camargo, Stefany Vaz, Popó",BR,Michael Ukstin,55,"portuguese, pt",Máquina da Fama,Sbt,BR,Game-show,Sbt,Máquina da Fama,2013,tt5224000,maquina fama,maquina fama,sbt,sbt,michael ukstin,maquina fama
23819,95f26c1efc4f493ca7e2032dd81d963f,"Sem Lenço, Sem Documento",Series,https://globoplay.globo.com/sem-lenco-sem-documento/t/NzYQP6hMSG,1,6,1,"Cléa Simões, Ricardo Blat, Tião D'Ávila, Ivan Setta, Ana Helena Berenger, Marilu Bueno, Jaime Barcellos, Ilva Niño, Cidinha Milan, Tony Ferreira, Isabel Ribeiro, Irma Álvarez, Christiane Torloni, Gracinda Freire, Sebastião Vasconcelos, Jonas Bloch, Ana Braga, Bruna Lombardi, Ney Latorraca, Arlete Salles",BR,Dennis Carvalho,32,"portuguese, pt","Sem Lenço, Sem Documento",,BR,Drama,,"Sem Lenço, Sem Documento",1977,tt0209791,sem lenco sem documento,sem lenco sem documento,,,dennis carvalho,sem lenco sem documento
23926,7bdd309e3e15a3cb428a46297a96af77,Hoje é Dia das Crianças,Series,https://globoplay.globo.com/hoje-e-dia-das-criancas/t/KT6pZ16qPw,0,0,,"Lua Miranda, Zezé Motta, Tony Tornado, Aisha Jambo, Duda Beat, MC Cabelinho, IZA, Jão, Nattan, Luan Pereira, Priscilla, Péricles, Pedro Sampaio",BR,Raoni Carneiro,0,portuguese,Hoje é Dia Das Crianças,,BR,Musical,,Hoje é Dia Das Crianças,2024,tt33886372,hoje dia criancas,hoje dia criancas,,,raoni carneiro,hoje dia criancas
156,a66a35ca1601749289b7133d4be96337,90 Dias Para Casar – Seguindo Colt e Larissa,Series,https://app.primevideo.com/detail?gti=amzn1.dv.gti.692eb419-c439-4f01-9ed5-0707a340d3c7,1,22,1,,BR,,0,,90 Dias Para Casar: Seguindo Colt e Larissa,,BR,,,90 Dias Para Casar: Seguindo Colt e Larissa,2022,,90 dias para casar seguindo colt larissa,90 dias para casar seguindo colt larissa,,,,90 dias para casar seguindo colt larissa


,BB UID,Platform Title,Type,Deeplink,Seasons,Episodes,Season Numbers,BB Cast,BB Countries,BB Directors,BB Duration,BB Languages,BB Original Title,BB Primary Company,BB Primary Country,BB Primary Genre,BB Production Companies,BB Title,BB Year,IMDb ID,Platform Title_processado,BB Original Title_processado,BB Primary Company_processado,BB Production Companies_processado,BB Directors_processado,BB Title_processado
38267,0d31c9770fbfeec25acc1f9f77aa3dd0,O Abismo entre Nós,Movie,https://www.vivoplay.com.br/details/movie/o-abismo-entre-nos-26144390,0,0,,,,,83,,,,,Suspense,,O Abismo entre Nós,2019,,abismo entre nos,,,,,abismo entre nos
30782,a1fb0c8c07baec9d2879f8320b036ee7,Coleção II Guerra Mundial,Movie,https://www.netmovies.com.br/media/movies/380272/colecao-ii-guerra-mundial-batalha-da-china,0,0,,,,"Frank Capra,Anatole Litvak",62,,,,,War,,Coleção II Guerra Mundial,1944,,colecao 2 guerra mundial,,,,frank capra anatole litvak,colecao 2 guerra mundial
9745,d7b4a45b90e7795dd60738ee1d9e185e,US ARMY - CENAS DO EQUIPAMENTO MILITAR DE DEZ ANOS ATRÁS,Movie,http://bcc.cinemateca.org.br/filmes/449129,0,0,,,,,3,,,,,,,US ARMY - CENAS DO EQUIPAMENTO MILITAR DE DEZ ANOS ATRÁS,1968,,us army cenas equipamento militar de dez anos atras,,,,,us army cenas equipamento militar de dez anos atras
21137,240022fe2d931588b71819454eb80a65,Infused Tea from the Motherland,Movie,https://watch.eventive.org/seeusonfilm2024/play/66d238d8759b43003223c659/66d0fb12c328d4060b774bd7,0,0,,"Zusi Airhiavbere, Reema Dazeé, Rylan Gray, Bobby Haynes, Tiffany Jana, Ellice McCoy, Volita Russel, LaKesshia Slaughter, Saliho Touré",US,MacKenzie Galloway,8,english,Infused Tea from the Motherland,,US,Short,,Infused Tea from the Motherland,2023,tt28639468,infused tea from motherland,infused tea from motherland,,,mackenzie galloway,infused tea from motherland
6372,a4767e270d19889549f6b7aa429c388c,Rangers x Club Brugge,Movie,https://app.primevideo.com/detail?gti=amzn1.dv.gti.e7385551-7e05-452c-9452-57c67c9ece0a,0,0,,,,,106,,,,,,,,2025,,rangers 10 club brugge,,,,,


,BB UID,Platform Title,Type,Deeplink,Seasons,Episodes,Season Numbers,BB Cast,BB Countries,BB Directors,BB Duration,BB Languages,BB Original Title,BB Primary Company,BB Primary Country,BB Primary Genre,BB Production Companies,BB Title,BB Year,IMDb ID,Platform Title_processado,BB Original Title_processado,BB Primary Company_processado,BB Production Companies_processado,BB Directors_processado,BB Title_processado
37936,143cc6a1e417e0b0ec4cda899c46c708,Perdido entre Lobos,Series,https://www.vivoplay.com.br/details/serie/perdido-entre-lobos-40643784,1,1,2,,,,0,,,,,Reality-tv,,Perdido entre Lobos,2020,,perdido entre lobos,,,,,perdido entre lobos
37857,b0b7c25f6913d00ea351ca64400604a3,Donkerbos: Floresta Obscura,Series,https://www.vivoplay.com.br/details/serie/donkerbos-floresta-obscura-40161058,1,8,1,,,,0,,,,,Suspense,,Donkerbos: Floresta Obscura,2022,,donkerbos floresta obscura,,,,,donkerbos floresta obscura
35684,ff286c9ec75d40aa2642c28c1d7b96a1,"Viola, Minha Viola",Series,https://painel.play.uol.com.br/series/3978-viola-minha-viola,1,48,5,,,Lucas Rochetti,0,,,,,Musical,,"Viola, Minha Viola",2007,,viola minha viola,,,,lucas rochetti,viola minha viola
29688,263f7cb080a427818ae0f043642ddf6a,Mic The Snare,Series,https://nebula.tv/micthesnare,0,0,,,,,0,,,,,,,Mic The Snare,0,,mic snare,,,,,mic snare
11380,c842ff74469091e3ab5861fcb56ed79d,COSTA E SILVA CONCEDE ENTREVISTA,Series,http://bcc.cinemateca.org.br/filmes/447348,1,0,1968,,,,0,,,,,,,COSTA E SILVA CONCEDE ENTREVISTA,1968,,costa silva concede entrevista,,,,,costa silva concede entrevista


### 3.3) Criação e tratamento dos campos de Ano de Produção

In [37]:
# Desativar warnings
warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)

# Aplicando as funções e criando as novas colunas nos DataFrames SAD
for df in [filmesBR_SAD_limpo, seriesBR_SAD_limpo]:
    df['Ano Inicial'] = df['Ano de Produção'].apply(extract_ano_inicial)
    df['Ano Final'] = df['Ano de Produção'].apply(extract_ano_final)

In [38]:
for df in [filmesBR_BB_limpo, seriesBR_BB_limpo]:
    df['BB Year'] = df['BB Year'].fillna(0).astype(int)
    
# Reativar warnings
warnings.simplefilter(action='default', category=pd.errors.SettingWithCopyWarning)


In [39]:
# A) Conversão de tipos numéricos (SAD) com validação
cols_int_sad = ['Ano Inicial',	'Ano Final', 'Nº de Episódios'] 

def to_nullable_int(series, nome):
    s = pd.to_numeric(series, errors='coerce')
    frac = (s % 1).fillna(0)
    # valida: só aceita inteiros exatos (fração = 0)
    if (frac != 0).any():
        # mostra exemplos problemáticos e aborta
        bad = series[(frac != 0)]
        raise ValueError(f"Coluna '{nome}' contém valores não-inteiros, ex.: {bad.head(10).tolist()}")
    return s.astype('Int64')

for c in cols_int_sad:
    if c in filmesBR_SAD_limpo.columns:
        filmesBR_SAD_limpo[c] = to_nullable_int(filmesBR_SAD_limpo[c], c)
    if c in seriesBR_SAD_limpo.columns:
        seriesBR_SAD_limpo[c]  = to_nullable_int(seriesBR_SAD_limpo[c], c)


In [40]:
# ================================================================
# 3.3 — Materialização de features para Matching (pré-cálculo)
# ================================================================

def _prod_tokens_union(row):
    s1 = row['BB Primary Company_processado']
    s2 = row['BB Production Companies_processado']
    toks = set()
    if isinstance(s1, str) and s1.strip():
        toks |= tokens_empresas(s1)
    if isinstance(s2, str) and s2.strip() and s2 != s1:
        toks |= tokens_empresas(s2)
    return toks

# SAD (filmes e séries)
for df in [filmesBR_SAD_limpo, seriesBR_SAD_limpo]:
    df['dir_set_sad']      = df['Diretor'].apply(surnames_from_raw)             # usa campo RAW (separado por vírgula)
    df['prod_tokens_sad']  = df['Produtora_processada'].apply(tokens_empresas)  # já normalizada

# BB (filmes/séries nacionais e estrangeiros)
for df in [filmesBR_BB_limpo, seriesBR_BB_limpo, filmesEstr_BB_limpo, seriesEstr_BB_limpo]:
    df['dir_set_bb']       = df['BB Directors'].apply(surnames_from_raw)        # usa campo RAW (separado por vírgula)
    df['prod_tokens_bb']   = df.apply(_prod_tokens_union, axis=1)               # união: Primary + Companies (se diferentes)


### 3.5) Verificação dos resultados após transformações

In [41]:
print(filmesBR_SAD_limpo.info())  # Resumo dos atributos
print("\nResumo Estatístico:")
print(filmesBR_SAD_limpo.describe())  # Resumo estatístico

<class 'pandas.core.frame.DataFrame'>
Index: 47791 entries, 0 to 59907
Data columns (total 19 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Nº CPB                      47791 non-null  object 
 1   Título Original             47790 non-null  object 
 2   Título Alternativo          1 non-null      object 
 3   Classificação               47791 non-null  object 
 4   Gênero                      47791 non-null  object 
 5   Organização Temporal        47791 non-null  object 
 6   Duração Obra                47791 non-null  float64
 7   Nº de Episódios             4327 non-null   Int64  
 8   Ano de Produção             47791 non-null  object 
 9   Produtora                   47783 non-null  object 
 10  Diretor                     47790 non-null  object 
 11  País de Origem              47374 non-null  object 
 12  Título Original_processado  47790 non-null  object 
 13  Diretor_processado          47790 no

In [42]:
print(seriesBR_SAD_limpo.info())  # Resumo dos atributos
print("\nResumo Estatístico:")
print(seriesBR_SAD_limpo.describe())  # Resumo estatístico

<class 'pandas.core.frame.DataFrame'>
Index: 12120 entries, 25 to 59910
Data columns (total 19 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Nº CPB                      12120 non-null  object 
 1   Título Original             12120 non-null  object 
 2   Título Alternativo          0 non-null      object 
 3   Classificação               12120 non-null  object 
 4   Gênero                      12120 non-null  object 
 5   Organização Temporal        12120 non-null  object 
 6   Duração Obra                12120 non-null  float64
 7   Nº de Episódios             12099 non-null  Int64  
 8   Ano de Produção             12118 non-null  object 
 9   Produtora                   12117 non-null  object 
 10  Diretor                     12120 non-null  object 
 11  País de Origem              12096 non-null  object 
 12  Título Original_processado  12120 non-null  object 
 13  Diretor_processado          12120 n

In [43]:
print(filmesBR_BB_limpo.info())  # Resumo dos atributos
print("\nResumo Estatístico:")
print(filmesBR_BB_limpo.describe())  # Resumo estatístico

<class 'pandas.core.frame.DataFrame'>
Index: 816 entries, 53 to 39345
Data columns (total 28 columns):
 #   Column                              Non-Null Count  Dtype 
---  ------                              --------------  ----- 
 0   BB UID                              816 non-null    object
 1   Platform Title                      816 non-null    object
 2   Type                                816 non-null    object
 3   Deeplink                            816 non-null    object
 4   Seasons                             816 non-null    int64 
 5   Episodes                            816 non-null    int64 
 6   Season Numbers                      816 non-null    object
 7   BB Cast                             816 non-null    object
 8   BB Countries                        816 non-null    object
 9   BB Directors                        816 non-null    object
 10  BB Duration                         816 non-null    int64 
 11  BB Languages                        816 non-null    object
 

In [44]:
print(seriesBR_BB_limpo.info())  # Resumo dos atributos
print("\nResumo Estatístico:")
print(seriesBR_BB_limpo.describe())  # Resumo estatístico

<class 'pandas.core.frame.DataFrame'>
Index: 361 entries, 156 to 37993
Data columns (total 28 columns):
 #   Column                              Non-Null Count  Dtype 
---  ------                              --------------  ----- 
 0   BB UID                              361 non-null    object
 1   Platform Title                      361 non-null    object
 2   Type                                361 non-null    object
 3   Deeplink                            361 non-null    object
 4   Seasons                             361 non-null    int64 
 5   Episodes                            361 non-null    int64 
 6   Season Numbers                      361 non-null    object
 7   BB Cast                             361 non-null    object
 8   BB Countries                        361 non-null    object
 9   BB Directors                        361 non-null    object
 10  BB Duration                         361 non-null    int64 
 11  BB Languages                        361 non-null    object


In [45]:
print(filmesEstr_BB_limpo.info())  # Resumo dos atributos
print("\nResumo Estatístico:")
print(filmesEstr_BB_limpo.describe())  # Resumo estatístico


<class 'pandas.core.frame.DataFrame'>
Index: 33371 entries, 0 to 39416
Data columns (total 28 columns):
 #   Column                              Non-Null Count  Dtype 
---  ------                              --------------  ----- 
 0   BB UID                              33371 non-null  object
 1   Platform Title                      33371 non-null  object
 2   Type                                33371 non-null  object
 3   Deeplink                            33371 non-null  object
 4   Seasons                             33371 non-null  int64 
 5   Episodes                            33371 non-null  int64 
 6   Season Numbers                      33371 non-null  object
 7   BB Cast                             33371 non-null  object
 8   BB Countries                        33371 non-null  object
 9   BB Directors                        33371 non-null  object
 10  BB Duration                         33371 non-null  int64 
 11  BB Languages                        33371 non-null  object


In [46]:
print(seriesEstr_BB_limpo.info())  # Resumo dos atributos
print("\nResumo Estatístico:")
print(seriesEstr_BB_limpo.describe())  # Resumo estatístico


<class 'pandas.core.frame.DataFrame'>
Index: 4869 entries, 26 to 38675
Data columns (total 28 columns):
 #   Column                              Non-Null Count  Dtype 
---  ------                              --------------  ----- 
 0   BB UID                              4869 non-null   object
 1   Platform Title                      4869 non-null   object
 2   Type                                4869 non-null   object
 3   Deeplink                            4869 non-null   object
 4   Seasons                             4869 non-null   int64 
 5   Episodes                            4869 non-null   int64 
 6   Season Numbers                      4869 non-null   object
 7   BB Cast                             4869 non-null   object
 8   BB Countries                        4869 non-null   object
 9   BB Directors                        4869 non-null   object
 10  BB Duration                         4869 non-null   int64 
 11  BB Languages                        4869 non-null   object


### 3.6) Salva bases Limpas para importação futura

In [47]:
# Nome do arquivo de saída
arquivo = 'Pre-processamento bases limpas.xlsx'

# Usando ExcelWriter para salvar os DataFrames em abas diferentes
with pd.ExcelWriter(arquivo, engine='openpyxl') as writer:
    # Escrevendo cada DataFrame em uma aba separada
    filmesBR_SAD_limpo.to_excel(writer, sheet_name='Filmes SAD', index=False)
    seriesBR_SAD_limpo.to_excel(writer, sheet_name='Séries SAD', index=False)
    filmesBR_BB_limpo.to_excel(writer, sheet_name='Filmes BR BB', index=False)
    seriesBR_BB_limpo.to_excel(writer, sheet_name='Séries BR BB', index=False)
    filmesEstr_BB_limpo.to_excel(writer, sheet_name='Filmes Estr BB', index=False)
    seriesEstr_BB_limpo.to_excel(writer, sheet_name='Séries Estr BB', index=False)


In [48]:
# Recupera bases limpas
arquivo = 'Pre-processamento bases limpas.xlsx'
filmesBR_SAD_limpo = pd.read_excel(arquivo, sheet_name='Filmes SAD')
seriesBR_SAD_limpo = pd.read_excel(arquivo, sheet_name='Séries SAD')
filmesBR_BB_limpo = pd.read_excel(arquivo, sheet_name='Filmes BR BB')
seriesBR_BB_limpo = pd.read_excel(arquivo, sheet_name='Séries BR BB')
filmesEstr_BB_limpo = pd.read_excel(arquivo, sheet_name='Filmes Estr BB')
seriesEstr_BB_limpo = pd.read_excel(arquivo, sheet_name='Séries Estr BB')

In [49]:
def _fix_headers(df):
    df.columns = (df.columns.astype(str)
                  .str.replace('\u00A0', ' ', regex=False)   # NBSP
                  .str.replace('\ufeff', '', regex=False)    # BOM
                  .str.replace(r'\s+', ' ', regex=True)      # espaços duplicados
                  .str.strip())
    return df

for df in [filmesBR_BB_limpo, seriesBR_BB_limpo, filmesEstr_BB_limpo, seriesEstr_BB_limpo,
           filmesBR_SAD_limpo, seriesBR_SAD_limpo]:
    _fix_headers(df)


In [50]:
required_bb = [
    'BB UID','BB Year','BB Directors','BB Primary Company','BB Production Companies',
    'BB Title_processado','BB Original Title_processado','Platform Title_processado'
]

for name, d in {
    'filmesBR_BB_limpo': filmesBR_BB_limpo,
    'seriesBR_BB_limpo': seriesBR_BB_limpo,
    'filmesEstr_BB_limpo': filmesEstr_BB_limpo,
    'seriesEstr_BB_limpo': seriesEstr_BB_limpo,
}.items():
    missing = [c for c in required_bb if c not in d.columns]
    if missing:
        print(f"{name} faltando:", missing)


## 4) Busca de Correspondências exatas de Título, Diretor e Ano

### 4.1) Definição de funções de busca

In [51]:
# ================================================================
# 4.1 Funções — motor de match limpo (título =, ano ±2, diretor obrigatório)
# ================================================================

ANO_TOL = 2  # tolerância

def _cpb_col(df):
    return 'Nº CPB' if 'Nº CPB' in df.columns else ('N° CPB' if 'N° CPB' in df.columns else 'Nº CPB')

def build_bb_title_keys(df_bb):
    title_cols = [
        'BB Title_processado',
        'BB Original Title_processado',
        'Platform Title_processado'
    ]
    keep = [
        'BB UID','BB Year','BB Directors',
        'BB Primary Company','BB Production Companies',
        # >>> campos processados que faltavam <<<
        'BB Primary Company_processado','BB Production Companies_processado',
        'BB Title','BB Original Title','Platform Title'
    ]
    m = df_bb[keep + title_cols].melt(
        id_vars=keep,
        value_vars=title_cols,
        var_name='title_source',
        value_name='BB_title_key'
    )
    m = m[m['BB_title_key'].notna() & (m['BB_title_key'] != '')]
    m = m.drop_duplicates(['BB UID','BB_title_key'], keep='first')
    return m


def match_pair_min(df_sad, df_bb, categoria):
    cpb = _cpb_col(df_sad)

    # 1) título
    bbk = build_bb_title_keys(df_bb)
    sadk = df_sad[[cpb, 'Título Original','Título Original_processado',
                   'Diretor','Ano Inicial','Ano Final','Produtora']].copy()
    cand = bbk.merge(sadk, left_on='BB_title_key', right_on='Título Original_processado', how='inner')

    # 2) ano ±2
    cand = cand.dropna(subset=['BB Year','Ano Inicial']).copy()
    cand['Ano Final'] = cand['Ano Final'].fillna(cand['Ano Inicial'])
    mask_ano = (
        (cand['BB Year'].astype(int) >= (cand['Ano Inicial'].astype(int) - ANO_TOL)) &
        (cand['BB Year'].astype(int) <= (cand['Ano Final'].astype(int)   + ANO_TOL))
    )
    cand = cand[mask_ano]

    # 3) diretor obrigatório
    a = cand['BB Directors'].apply(surnames_from_raw)
    b = cand['Diretor'].apply(surnames_from_raw)
    cand['dir_overlap'] = [sorted(list(x & y)) for x, y in zip(a, b)]
    cand = cand[cand['dir_overlap'].map(len) > 0].copy()

    # 4) saída
    out = cand[[cpb, 'BB UID',
                'Título Original',
                'BB Title','BB Original Title','Platform Title',
                'Diretor','BB Directors',
                'Ano Inicial','Ano Final','BB Year',
                'Produtora','BB Primary Company','BB Production Companies']].copy()

    out['categoria']  = categoria
    out['match_rule'] = 'titulo+ano+diretor'
    out = out.rename(columns={'BB UID':'UID', 'BB Title':'Title',
                              'BB Directors':'Directors', 'BB Year':'Year',
                              cpb: cpb})
    return out


In [52]:
# ================================================================
# 4.1b — regra 2: título =, ano ±2, PRODUTORA (tokens) obrigatória
# ================================================================

def _bb_prod_tokens_row(r):
    s1 = str(r.get('BB Primary Company_processado', '') or '').strip()
    s2 = str(r.get('BB Production Companies_processado', '') or '').strip()
    toks = set()
    if s1:
        toks |= tokens_empresas(s1)
    if s2 and s2 != s1:
        toks |= tokens_empresas(s2)
    return toks

def match_pair_produtora(df_sad, df_bb, categoria):
    cpb = _cpb_col(df_sad)

    # 1) título
    bbk = build_bb_title_keys(df_bb)
    sadk = df_sad[[cpb, 'Título Original','Título Original_processado',
                   'Produtora','Produtora_processada',
                   'Ano Inicial','Ano Final',
                   'Diretor']].copy()  # diretor só para visualização/auditoria
    cand = bbk.merge(sadk, left_on='BB_title_key', right_on='Título Original_processado', how='inner')

    # 2) ano ±2
    cand = cand.dropna(subset=['BB Year','Ano Inicial']).copy()
    cand['Ano Final'] = cand['Ano Final'].fillna(cand['Ano Inicial'])
    mask_ano = (
        (cand['BB Year'].astype(int) >= (cand['Ano Inicial'].astype(int) - ANO_TOL)) &
        (cand['BB Year'].astype(int) <= (cand['Ano Final'].astype(int)   + ANO_TOL))
    )
    cand = cand[mask_ano]

    # 3) PRODUTORA — overlap de tokens
    cand['prod_bb']  = cand.apply(_bb_prod_tokens_row, axis=1)
    cand['prod_sad'] = cand['Produtora_processada'].apply(tokens_empresas)
    cand['prod_overlap'] = [sorted(list(x & y)) for x, y in zip(cand['prod_bb'], cand['prod_sad'])]
    cand = cand[cand['prod_overlap'].map(len) > 0].copy()

    # 4) saída
    out = cand[[cpb, 'BB UID',
                'Título Original',
                'BB Title','BB Original Title','Platform Title',
                'Diretor','BB Directors',
                'Ano Inicial','Ano Final','BB Year',
                'Produtora','BB Primary Company','BB Production Companies']].copy()

    out['categoria']  = categoria
    out['match_rule'] = 'titulo+ano+produtora'
    out = out.rename(columns={'BB UID':'UID', 'BB Title':'Title',
                              'BB Directors':'Directors', 'BB Year':'Year',
                              cpb: cpb})
    return out


In [53]:
# ================================================================
# 4.2 — Executar PRODUTORA e unir com DIRETOR (prioridade ao DIRETOR)
# ================================================================

# Se você já rodou a 1ª rodada e tem out_* da regra diretor, mantenha:
dir_filmes_BR   = match_pair_min(filmesBR_SAD_limpo,  filmesBR_BB_limpo,   'filmes_BR')
dir_series_BR   = match_pair_min(seriesBR_SAD_limpo,  seriesBR_BB_limpo,   'series_BR')
dir_filmes_EST  = match_pair_min(filmesBR_SAD_limpo,  filmesEstr_BB_limpo, 'filmes_EST')
dir_series_EST  = match_pair_min(seriesBR_SAD_limpo,  seriesEstr_BB_limpo, 'series_EST')

matches_dir = pd.concat([dir_filmes_BR, dir_series_BR, dir_filmes_EST, dir_series_EST],
                        ignore_index=True)

# Filtrar os BB ainda não casados para a rodada PRODUTORA (opcional, mas recomendado)
uids_casados = set(matches_dir['UID'].unique())

filmesBR_BB_rest   = filmesBR_BB_limpo[~filmesBR_BB_limpo['BB UID'].isin(uids_casados)]
seriesBR_BB_rest   = seriesBR_BB_limpo[~seriesBR_BB_limpo['BB UID'].isin(uids_casados)]
filmesEstr_BB_rest = filmesEstr_BB_limpo[~filmesEstr_BB_limpo['BB UID'].isin(uids_casados)]
seriesEstr_BB_rest = seriesEstr_BB_limpo[~seriesEstr_BB_limpo['BB UID'].isin(uids_casados)]

prod_filmes_BR   = match_pair_produtora(filmesBR_SAD_limpo,  filmesBR_BB_rest,   'filmes_BR')
prod_series_BR   = match_pair_produtora(seriesBR_SAD_limpo,  seriesBR_BB_rest,   'series_BR')
prod_filmes_EST  = match_pair_produtora(filmesBR_SAD_limpo,  filmesEstr_BB_rest, 'filmes_EST')
prod_series_EST  = match_pair_produtora(seriesBR_SAD_limpo,  seriesEstr_BB_rest, 'series_EST')

matches_prod = pd.concat([prod_filmes_BR, prod_series_BR, prod_filmes_EST, prod_series_EST],
                         ignore_index=True)

# União final: DIRETOR tem prioridade; depois PRODUTORA completa o que faltou
# (se preferir, pode trocar 'keep="first"' por 'keep="last"' invertendo a prioridade)
cpb_col = 'Nº CPB' if 'Nº CPB' in filmesBR_SAD_limpo.columns else 'N° CPB'
matches_all = pd.concat([matches_dir, matches_prod], ignore_index=True)
matches_all = matches_all.drop_duplicates(subset=['UID', cpb_col], keep='first')

# Contagens
print("Contagens por categoria (após união):")
print(matches_all.groupby(['categoria','match_rule']).size().unstack(fill_value=0))
print("\nUIDs únicos casados (total):", matches_all['UID'].nunique())
print("CPBs únicos casados (total):", matches_all[cpb_col].nunique())

# Amostra para inspeção
display(matches_all.head(12))


Contagens por categoria (após união):
match_rule  titulo+ano+diretor  titulo+ano+produtora
categoria                                           
filmes_BR                  303                   135
filmes_EST                 336                     5
series_BR                   69                    38
series_EST                 117                     0

UIDs únicos casados (total): 955
CPBs únicos casados (total): 970


,Nº CPB,UID,Título Original,Title,BB Original Title,Platform Title,Diretor,Directors,Ano Inicial,Ano Final,Year,Produtora,BB Primary Company,BB Production Companies,categoria,match_rule
0,B0901097600000,97b6a71422a7ef096252cbd035467c20,BREGA S/A,Brega S/A,Brega S/A,Brega S/A,"VLADIMIR AUGUSTO TAVARES DA CUNHA, GUSTAVO SIL...","Gustavo Godinho, Vladimir Cunha",2009.0,2009.0,2009,COMPANHIA AMAZONICA DE FILMES S/S LTDA.,NaN,NaN,filmes_BR,titulo+ano+diretor
1,B1001326900000,e3c4ad742c334a1465dddab9e19f7d57,SOLDADOS DA BORRACHA,Soldados da Borracha,Soldados da Borracha,Soldados da Borracha,CESAR AUGUSTO GARCIA LIMA,Cesar Garcia Lima,2010.0,2010.0,2011,MODO OPERANTE PRODUÇÕES CULTURAIS LTDA. ME,NaN,NaN,filmes_BR,titulo+ano+diretor
2,B1900033100000,faa5ffb612e89a155e1ce9e0a64c702e,JAÇANÃ,Jaçanã,Jaçanã,Jaçanã,"YASMIN ALVES PINTO, LARISSA VALIM MACHADO, CLA...",Clarissa Virmond,2018.0,2018.0,2018,GALO DE BRIGA PRODUÇÕES LTDA - EPP,Galo De Briga Filmes,Galo De Briga Filmes,filmes_BR,titulo+ano+diretor
3,B2300123500000,00e30d6ec6bac584b9332b97de4210c7,MADAME DUROCHER,Madame Durocher,Madame Durocher,Madame Durocher,"DIEGO ANDRADE RIVAS GUTIERREZ, ANDRE SALLES OL...","Dida Andrade, Andradina Azevedo",2023.0,2023.0,2024,CLARO VIDEO LLC | NEXUS CINEMA E VÍDEO LTDA,Nexus Cinema E Vídeo,Nexus Cinema E Vídeo,filmes_BR,titulo+ano+diretor
4,B2100222400000,211f361c4fb5ea789c9292e71e8d70e0,ACHADOS NÃO PROCURADOS,Achados Não Procurados,Achados Não Procurados,Achados Não Procurados,FABIANA HEILMANN PENNA,Fabi Penna,2021.0,2021.0,2021,ADALBERTO PENNA PRODUCOES CINEMATOGRAFICAS LTDA,Penna Filho Produções,Penna Filho Produções,filmes_BR,titulo+ano+diretor
5,B2400072200000,97669b5e94229249b28b4d65bb0299f3,JUZÉ,Juzé,Juzé,Juzé,RAQUEL VANESSA HERRERO GARCIA,Raquel Garcia,2024.0,2024.0,2023,ART&CIA ANIMATION SCHOOL LTDA,Animare Studio,Animare Studio,filmes_BR,titulo+ano+diretor
6,B2300330000000,de3daddb47ce003219492bf8acd8126e,ENTRELINHAS,Entrelinhas,Entrelinhas,Entrelinhas,AUGUSTINHO PASKO,Guto Pasko,2023.0,2023.0,2024,GP7 CINEMA LTDA,Gp7 Cinema,Gp7 Cinema,filmes_BR,titulo+ano+diretor
7,B1101445100000,da9f9b4be40c053cdf568aa8580b88db,ROBERTO MARINHO - O SENHOR DO SEU TEMPO,Roberto Marinho - O Senhor do Seu Tempo,Roberto Marinho - O Senhor do Seu Tempo,Grandes Brasileiros - Roberto Marinho - O Senh...,DERMEVAL COUTINHO NETTO,"Dermeval Netto, Rozane Braga",2011.0,2011.0,2011,"FBL E ASSOCIADOS, COMUNICAÇÕES LTDA",NaN,NaN,filmes_BR,titulo+ano+diretor
8,B2400311300000,7b4b97da79b28f424dc7e81d326d8daa,"PERFEKTA, UMA AVENTURA DA ESCOLA DE GÊNIOS",Perfekta: Uma Aventura da Escola de Gênios,Perfekta: Uma Aventura da Escola de Gênios,"Perfekta, Uma Aventura da Escola de Gênios",JOAO DANIEL SEQUEIRA TIKHOMIROFF,João Daniel Tikhomiroff,2024.0,2024.0,2024,GLOBO COMUNICAÇÃO E PARTICIPAÇÕES S/A | RSMTS ...,Globo Filmes,"Globo Filmes, Mixer Films, Gloob",filmes_BR,titulo+ano+diretor
9,B2300016300000,1ab4382a45e4f3f6a41184a3a55492cc,A LEI E O MEDO,A Lei e o Medo,A Lei e o Medo,A Lei e o Medo,HERMES FILHO LEAL,Hermes Leal,2022.0,2022.0,2020,PRODUTORA DE FILMES H.L. LTDA-ME,NaN,NaN,filmes_BR,titulo+ano+diretor


In [54]:
# ================================================================
# 4.3 Resultados — contagens e amostra
# ================================================================

print("Pares por categoria:")
print(matches_all['categoria'].value_counts(), "\n")

print("UIDs únicos casados:", matches_all['UID'].nunique())
print("CPBs únicos casados:", matches_all['Nº CPB'].nunique(), "\n")  # ajuste p/ 'N° CPB' se for o seu caso

display(matches_all.head(12))


Pares por categoria:
categoria
filmes_BR     438
filmes_EST    341
series_EST    117
series_BR     107
Name: count, dtype: int64 

UIDs únicos casados: 955
CPBs únicos casados: 970 



,Nº CPB,UID,Título Original,Title,BB Original Title,Platform Title,Diretor,Directors,Ano Inicial,Ano Final,Year,Produtora,BB Primary Company,BB Production Companies,categoria,match_rule
0,B0901097600000,97b6a71422a7ef096252cbd035467c20,BREGA S/A,Brega S/A,Brega S/A,Brega S/A,"VLADIMIR AUGUSTO TAVARES DA CUNHA, GUSTAVO SIL...","Gustavo Godinho, Vladimir Cunha",2009.0,2009.0,2009,COMPANHIA AMAZONICA DE FILMES S/S LTDA.,NaN,NaN,filmes_BR,titulo+ano+diretor
1,B1001326900000,e3c4ad742c334a1465dddab9e19f7d57,SOLDADOS DA BORRACHA,Soldados da Borracha,Soldados da Borracha,Soldados da Borracha,CESAR AUGUSTO GARCIA LIMA,Cesar Garcia Lima,2010.0,2010.0,2011,MODO OPERANTE PRODUÇÕES CULTURAIS LTDA. ME,NaN,NaN,filmes_BR,titulo+ano+diretor
2,B1900033100000,faa5ffb612e89a155e1ce9e0a64c702e,JAÇANÃ,Jaçanã,Jaçanã,Jaçanã,"YASMIN ALVES PINTO, LARISSA VALIM MACHADO, CLA...",Clarissa Virmond,2018.0,2018.0,2018,GALO DE BRIGA PRODUÇÕES LTDA - EPP,Galo De Briga Filmes,Galo De Briga Filmes,filmes_BR,titulo+ano+diretor
3,B2300123500000,00e30d6ec6bac584b9332b97de4210c7,MADAME DUROCHER,Madame Durocher,Madame Durocher,Madame Durocher,"DIEGO ANDRADE RIVAS GUTIERREZ, ANDRE SALLES OL...","Dida Andrade, Andradina Azevedo",2023.0,2023.0,2024,CLARO VIDEO LLC | NEXUS CINEMA E VÍDEO LTDA,Nexus Cinema E Vídeo,Nexus Cinema E Vídeo,filmes_BR,titulo+ano+diretor
4,B2100222400000,211f361c4fb5ea789c9292e71e8d70e0,ACHADOS NÃO PROCURADOS,Achados Não Procurados,Achados Não Procurados,Achados Não Procurados,FABIANA HEILMANN PENNA,Fabi Penna,2021.0,2021.0,2021,ADALBERTO PENNA PRODUCOES CINEMATOGRAFICAS LTDA,Penna Filho Produções,Penna Filho Produções,filmes_BR,titulo+ano+diretor
5,B2400072200000,97669b5e94229249b28b4d65bb0299f3,JUZÉ,Juzé,Juzé,Juzé,RAQUEL VANESSA HERRERO GARCIA,Raquel Garcia,2024.0,2024.0,2023,ART&CIA ANIMATION SCHOOL LTDA,Animare Studio,Animare Studio,filmes_BR,titulo+ano+diretor
6,B2300330000000,de3daddb47ce003219492bf8acd8126e,ENTRELINHAS,Entrelinhas,Entrelinhas,Entrelinhas,AUGUSTINHO PASKO,Guto Pasko,2023.0,2023.0,2024,GP7 CINEMA LTDA,Gp7 Cinema,Gp7 Cinema,filmes_BR,titulo+ano+diretor
7,B1101445100000,da9f9b4be40c053cdf568aa8580b88db,ROBERTO MARINHO - O SENHOR DO SEU TEMPO,Roberto Marinho - O Senhor do Seu Tempo,Roberto Marinho - O Senhor do Seu Tempo,Grandes Brasileiros - Roberto Marinho - O Senh...,DERMEVAL COUTINHO NETTO,"Dermeval Netto, Rozane Braga",2011.0,2011.0,2011,"FBL E ASSOCIADOS, COMUNICAÇÕES LTDA",NaN,NaN,filmes_BR,titulo+ano+diretor
8,B2400311300000,7b4b97da79b28f424dc7e81d326d8daa,"PERFEKTA, UMA AVENTURA DA ESCOLA DE GÊNIOS",Perfekta: Uma Aventura da Escola de Gênios,Perfekta: Uma Aventura da Escola de Gênios,"Perfekta, Uma Aventura da Escola de Gênios",JOAO DANIEL SEQUEIRA TIKHOMIROFF,João Daniel Tikhomiroff,2024.0,2024.0,2024,GLOBO COMUNICAÇÃO E PARTICIPAÇÕES S/A | RSMTS ...,Globo Filmes,"Globo Filmes, Mixer Films, Gloob",filmes_BR,titulo+ano+diretor
9,B2300016300000,1ab4382a45e4f3f6a41184a3a55492cc,A LEI E O MEDO,A Lei e o Medo,A Lei e o Medo,A Lei e o Medo,HERMES FILHO LEAL,Hermes Leal,2022.0,2022.0,2020,PRODUTORA DE FILMES H.L. LTDA-ME,NaN,NaN,filmes_BR,titulo+ano+diretor


In [55]:
# ================================================================
# Sumarização por categoria com PRODUTORA (+ ganhos novos)
# Requer: matches_dir, matches_prod, build_bb_title_keys, surnames_from_raw,
#         tokens_empresas, _bb_prod_tokens_row (definidas antes)
# ================================================================

def funil_uid_com_prod(df_sad, df_bb, categoria, matches_dir, matches_prod):
    # base = UIDs únicos do BB
    obras = df_bb['BB UID'].nunique()

    # --- título (sem ano) ---
    bbk  = build_bb_title_keys(df_bb)
    sadk = df_sad[['Título Original_processado', 'Diretor',
                   'Produtora_processada', 'Ano Inicial', 'Ano Final']].copy()
    e_title = bbk.merge(sadk, left_on='BB_title_key',
                        right_on='Título Original_processado', how='inner')

    uids_titulo = set(e_title['BB UID'].unique())
    n_titulo = len(uids_titulo)

    # --- diretor (sem ano) ---
    a = e_title['BB Directors'].apply(surnames_from_raw)
    b = e_title['Diretor'].apply(surnames_from_raw)
    dir_ok = [len(x & y) > 0 for x, y in zip(a, b)]
    uids_dir = set(e_title[dir_ok]['BB UID'].unique())
    n_dir = len(uids_dir)

    # --- produtora (sem ano) ---
    prod_bb  = e_title.apply(_bb_prod_tokens_row, axis=1)
    prod_sad = e_title['Produtora_processada'].apply(tokens_empresas)
    prod_ok  = [len(x & y) > 0 for x, y in zip(prod_bb, prod_sad)]
    uids_prod = set(e_title[prod_ok]['BB UID'].unique())
    n_prod = len(uids_prod)

    # --- finais por regra (já com ano) a partir das saídas de cada regra ---
    u_dir_final  = set(matches_dir.loc[matches_dir['categoria']==categoria, 'UID'].unique())
    u_prod_final = set(matches_prod.loc[matches_prod['categoria']==categoria, 'UID'].unique())

    final_total_uids   = u_dir_final | u_prod_final
    novos_via_produtor = u_prod_final - u_dir_final

    # conversões
    conv_total_titulo   = (n_titulo / obras * 100) if obras else 0.0
    conv_tit_dir        = (n_dir    / n_titulo * 100) if n_titulo else 0.0
    conv_tit_prod       = (n_prod   / n_titulo * 100) if n_titulo else 0.0
    conv_total          = (len(final_total_uids) / obras * 100) if obras else 0.0

    return pd.DataFrame([{
        'categoria': categoria,
        'obras': obras,
        'regra_titulo': n_titulo,
        'regra_diretor': n_dir,
        'regra_produtora': n_prod,
        'final_dir_ano': len(u_dir_final),
        'final_prod_ano': len(u_prod_final),
        'final_total': len(final_total_uids),
        'novos_via_produtora': len(novos_via_produtor),
        'conv_total_titulo_%': round(conv_total_titulo, 2),
        'conv_titulo→diretor_%': round(conv_tit_dir, 2),
        'conv_titulo→produtora_%': round(conv_tit_prod, 2),
        'conv_total_%': round(conv_total, 2),
    }])

    

In [56]:
# ---- Rodar por categoria ----
sum_f_br = funil_uid_com_prod(filmesBR_SAD_limpo,  filmesBR_BB_limpo,   'filmes_BR',  matches_dir, matches_prod)
sum_s_br = funil_uid_com_prod(seriesBR_SAD_limpo,  seriesBR_BB_limpo,   'series_BR',  matches_dir, matches_prod)
sum_f_es = funil_uid_com_prod(filmesBR_SAD_limpo,  filmesEstr_BB_limpo, 'filmes_EST', matches_dir, matches_prod)
sum_s_es = funil_uid_com_prod(seriesBR_SAD_limpo,  seriesEstr_BB_limpo, 'series_EST', matches_dir, matches_prod)

sum_cat = pd.concat([sum_f_br, sum_s_br, sum_f_es, sum_s_es], ignore_index=True)

# ---- Global (somando categorias) ----
glob = pd.DataFrame([{
    'obras':            sum_cat['obras'].sum(),
    'regra_titulo':     sum_cat['regra_titulo'].sum(),
    'regra_diretor':    sum_cat['regra_diretor'].sum(),
    'regra_produtora':  sum_cat['regra_produtora'].sum(),
    'final_dir_ano':    sum_cat['final_dir_ano'].sum(),
    'final_prod_ano':   sum_cat['final_prod_ano'].sum(),
    'final_total':      sum_cat['final_total'].sum(),
    'novos_via_produtora': sum_cat['novos_via_produtora'].sum()
}])

glob['conv_total_titulo_%']    = (glob['regra_titulo']   / glob['obras'] * 100).round(2)
glob['conv_titulo→diretor_%']  = (glob['regra_diretor']  / glob['regra_titulo'] * 100).round(2)
glob['conv_titulo→produtora_%']= (glob['regra_produtora']/ glob['regra_titulo'] * 100).round(2)
glob['conv_total_%']           = (glob['final_total']    / glob['obras'] * 100).round(2)

# ---- Exibir ----
fmt = {
    'conv_total_titulo_%':'{:.2f}%',
    'conv_titulo→diretor_%':'{:.2f}%',
    'conv_titulo→produtora_%':'{:.2f}%',
    'conv_total_%':'{:.2f}%'
}
print("Resumo por categoria (com produtora):")
display(sum_cat.style.format(fmt))

print("\nResumo global:")
display(glob.style.format(fmt))


Resumo por categoria (com produtora):


,categoria,obras,regra_titulo,regra_diretor,regra_produtora,final_dir_ano,final_prod_ano,final_total,novos_via_produtora,conv_total_titulo_%,conv_titulo→diretor_%,conv_titulo→produtora_%,conv_total_%
0,filmes_BR,816,622,316,360,302,133,435,133,76.23%,50.80%,57.88%,53.31%
1,series_BR,361,195,70,82,64,33,97,33,54.02%,35.90%,42.05%,26.87%
2,filmes_EST,33371,2087,382,14,335,5,340,5,6.25%,18.30%,0.67%,1.02%
3,series_EST,4869,396,101,1,83,0,83,0,8.13%,25.51%,0.25%,1.70%



Resumo global:


,obras,regra_titulo,regra_diretor,regra_produtora,final_dir_ano,final_prod_ano,final_total,novos_via_produtora,conv_total_titulo_%,conv_titulo→diretor_%,conv_titulo→produtora_%,conv_total_%
0,39417,3300,869,457,784,171,955,171,8.37%,26.33%,13.85%,2.42%


## 5 ) Exports para Excel

In [57]:
# ================================================================
# ETAPA 1 — Consolidar resultado final e salvar auditoria
# Requer já existirem: matches_dir, matches_prod
# ================================================================

# 1) escolher o nome da coluna CPB conforme sua base (Nº CPB ou N° CPB)
cpb_col = 'Nº CPB' if 'Nº CPB' in filmesBR_SAD_limpo.columns else 'N° CPB'

# 2) união (prioridade ao DIRETOR; o concat mantém a ordem: dir -> prod)
matches_all = pd.concat([matches_dir, matches_prod], ignore_index=True)
matches_all = matches_all.drop_duplicates(subset=['UID', cpb_col], keep='first')

# 3) ordenar e selecionar colunas para visão lado a lado
cols_auditoria = [
    cpb_col, 'UID',
    'Título Original', 'Title', 'BB Original Title', 'Platform Title',
    'Diretor', 'Directors',
    'Ano Inicial', 'Ano Final', 'Year',
    'Produtora', 'BB Primary Company', 'BB Production Companies',
    'categoria', 'match_rule'
]
# mantém colunas que existem; evita KeyError caso alguma não esteja presente
cols_auditoria = [c for c in cols_auditoria if c in matches_all.columns]

matches_all = matches_all[cols_auditoria].sort_values(
    by=['categoria', 'match_rule', 'Title' if 'Title' in matches_all.columns else cpb_col]
).reset_index(drop=True)

# 4) resumos
resumo_categoria = (
    matches_all
      .groupby('categoria')
      .agg(pares=('UID', 'size'),
           uids=('UID', 'nunique'),
           cpbs=(cpb_col, 'nunique'))
      .reset_index()
)

resumo_por_regra = (
    matches_all
      .groupby(['categoria', 'match_rule'])
      .size()
      .unstack(fill_value=0)
      .reset_index()
)

print("Resumo por categoria:")
display(resumo_categoria)

print("\nResumo por regra:")
display(resumo_por_regra)

# 5) salvar Excel de auditoria
arquivo_auditoria = 'Auditoria - matches consolidado.xlsx'
with pd.ExcelWriter(arquivo_auditoria, engine='openpyxl') as writer:
    matches_all.to_excel(writer, sheet_name='consolidado', index=False)
    resumo_categoria.to_excel(writer, sheet_name='resumo_categoria', index=False)
    resumo_por_regra.to_excel(writer, sheet_name='resumo_regra', index=False)

print(f"\nArquivo salvo: {arquivo_auditoria}")


Resumo por categoria:


,categoria,pares,uids,cpbs
0,filmes_BR,438,435,436
1,filmes_EST,341,340,330
2,series_BR,107,97,107
3,series_EST,117,83,106



Resumo por regra:


match_rule,categoria,titulo+ano+diretor,titulo+ano+produtora
0,filmes_BR,303,135
1,filmes_EST,336,5
2,series_BR,69,38
3,series_EST,117,0



Arquivo salvo: Auditoria - matches consolidado.xlsx


In [58]:
# ================================================================
# ETAPA 2 — Materializar estágios (por categoria) e exportar
# Requer: build_bb_title_keys, surnames_from_raw, tokens_empresas,
#         _bb_prod_tokens_row, e os DFs *_SAD_limpo / *_BB_limpo
# ================================================================

cpb_col = 'Nº CPB' if 'Nº CPB' in filmesBR_SAD_limpo.columns else 'N° CPB'
ANO_TOL = 2

def materializar_estagios(df_sad, df_bb, categoria):
    # 0) base por título
    bbk = build_bb_title_keys(df_bb)
    sadk = df_sad[[cpb_col, 'Título Original', 'Título Original_processado',
                   'Diretor', 'Produtora', 'Produtora_processada',
                   'Ano Inicial', 'Ano Final']].copy()
    base = bbk.merge(sadk, left_on='BB_title_key', right_on='Título Original_processado', how='inner')

    # estágio 1 — só TÍTULO
    cols_view = [cpb_col, 'BB UID', 'Título Original',
                 'BB Title','BB Original Title','Platform Title',
                 'Diretor','BB Directors',
                 'Produtora','BB Primary Company','BB Production Companies',
                 'Ano Inicial','Ano Final','BB Year']
    titulo_matched = base[cols_view].rename(columns={'BB UID':'UID', 'BB Title':'Title', 'BB Year':'Year'})

    # estágio 2 — TÍTULO + DIRETOR (sem ano)
    a = base['BB Directors'].apply(surnames_from_raw)
    b = base['Diretor'].apply(surnames_from_raw)
    dir_ok = [len(x & y) > 0 for x, y in zip(a, b)]
    titulo_diretor_matched = titulo_matched[dir_ok].copy()

    # estágio 3 — TÍTULO + PRODUTORA (sem ano)
    prod_bb  = base.apply(_bb_prod_tokens_row, axis=1)
    prod_sad = base['Produtora_processada'].apply(tokens_empresas)
    prod_ok  = [len(x & y) > 0 for x, y in zip(prod_bb, prod_sad)]
    titulo_produtora_matched = titulo_matched[prod_ok].copy()

    # helper ano ± tol
    def filtrar_ano(df):
        if df.empty: 
            return df
        df = df.dropna(subset=['Ano Inicial']).copy()
        df['Ano Final'] = df['Ano Final'].fillna(df['Ano Inicial'])
        mask = (
            (df['Year'].astype(int) >= (df['Ano Inicial'].astype(int) - ANO_TOL)) &
            (df['Year'].astype(int) <= (df['Ano Final'].astype(int)   + ANO_TOL))
        )
        return df[mask]

    # estágio 4 — FINAL (título + diretor + ano)
    final_dir_ano  = filtrar_ano(titulo_diretor_matched).copy()

    # estágio 5 — FINAL (título + produtora + ano)
    final_prod_ano = filtrar_ano(titulo_produtora_matched).copy()

    # metadados
    for df in [titulo_matched, titulo_diretor_matched, titulo_produtora_matched, final_dir_ano, final_prod_ano]:
        df['categoria'] = categoria

    return {
        'titulo_matched': titulo_matched,
        'titulo_diretor_matched': titulo_diretor_matched,
        'titulo_produtora_matched': titulo_produtora_matched,
        'final_dir_ano': final_dir_ano,
        'final_prod_ano': final_prod_ano
    }

# ---- rodar para os 4 pares ----
st_filmes_BR  = materializar_estagios(filmesBR_SAD_limpo,  filmesBR_BB_limpo,   'filmes_BR')
st_series_BR  = materializar_estagios(seriesBR_SAD_limpo,  seriesBR_BB_limpo,   'series_BR')
st_filmes_EST = materializar_estagios(filmesBR_SAD_limpo,  filmesEstr_BB_limpo, 'filmes_EST')
st_series_EST = materializar_estagios(seriesBR_SAD_limpo,  seriesEstr_BB_limpo, 'series_EST')

# ---- exportar
arq_estagios = 'Auditoria - estágios por categoria.xlsx'
with pd.ExcelWriter(arq_estagios, engine='openpyxl') as wr:
    for prefixo, st in [
        ('filmes_BR',  st_filmes_BR),
        ('series_BR',  st_series_BR),
        ('filmes_EST', st_filmes_EST),
        ('series_EST', st_series_EST),
    ]:
        st['titulo_matched'].to_excel(wr, sheet_name=f'{prefixo} - título', index=False)
        st['titulo_diretor_matched'].to_excel(wr, sheet_name=f'{prefixo} - título+diretor', index=False)
        st['titulo_produtora_matched'].to_excel(wr, sheet_name=f'{prefixo} - título+prod', index=False)
        st['final_dir_ano'].to_excel(wr, sheet_name=f'{prefixo} - final dir', index=False)
        st['final_prod_ano'].to_excel(wr, sheet_name=f'{prefixo} - final prod', index=False)

print(f'Arquivo salvo: {arq_estagios}')


Arquivo salvo: Auditoria - estágios por categoria.xlsx


### 4.3) Elimina plataformas sem interesse em buscas

In [59]:
# DF completo e coluna de plataforma para o join
df_completo_plataformas = Base_BB
col_plataforma = 'Platform Name'

# Plataformas a excluir (lista)
PLATAFORMAS_EXCLUIR = [
    '99 Media',    'AFA Play',    'ALTBalaji',    'American Indian Film Gallery',    'Amazon Prime Video',    'Anime Onegai',    'AppleTV',     'Apple TV+',    'Archivio Luce',    'AXN',    'BroadwayHD',
    'CINE.AR PLAY',    'Canela.TV',    'Cindie',    'Claro TV+',    'Claro Video',    ' CirqueConnect',    'Combate',    'Crunchyroll',    'Cultpix',    'Curiosity Stream',    'DaFilms',    'DAZN',
    'Dekkoo',    'Demand Africa',    'Digital Concert Hall',    'Digital Theatre',    'Disney+',    'DOCSVILLE',    'Eventive',    'F1 TV',    'Fanatiz',    'FIFA+',    'Filmbox+',    'Filmzie',
    'FlixOlé',    'Globoplay',    'Globe Player',    'GuideDoc',    'HENRI',    'HispanTV',    'History Hit',    'Hoichoi',    'IFI Archive Player',    'IndieFlix',    'iQIYI',    'IWantTFC',    'Kidoodle.TV',
    'KINOA.TV',    'KOCOWA+',    'KweliTV',    'Looke',    'Max',    'MagellanTV',    'Marquee TV',    'Means TV',    'Mercado Play',    'Met Opera on Demand',    'MLB.TV',    'MovieSaints',    'MUBI',
    'NBA League Pass',    'Nebula',    'Netflix',    'OCULTO.TV',    'Oldflix',    'OnDemandKorea',    'OperaVision',    'Paramount+',    'Plex',    'Pluto TV',    'Qello Concerts',    'Rakuten Viki',
    'Reel Short',    'Retina Latina',    'Revry',    'RT en Español',    'Selecta TV',    'ShemarooMe',    'Simply South',    'Sky+',    'Sony Channel',    'Tamanduá TV',    'Teatrix',    'Toon Goggles',
    'Troma NOW!',    'TV Caiçara',    'TVN Play',    'Umbra',    'UNIVER VIDEO',    'Universal+',    'Viddsee',    'Vivo Play',    'Watch',    'WOW Presents Plus',    'YouTube',    'YouTube Premium',    'Zee5',
]

# (opcional) conjunto para checagens/membership rápidas
PLATAFORMAS_EXCLUIR_SET = set(PLATAFORMAS_EXCLUIR)

print("Total plataformas a excluir:", len(PLATAFORMAS_EXCLUIR))


Total plataformas a excluir: 98


In [60]:
# ================================================================
# Filas de revisão: estrangeiras com country vazio, diretor presente,
# mapeando plataformas a partir do DF completo (não deduplicado)
# Parâmetros que você PRECISA definir antes:
#   PLATAFORMAS_EXCLUIR = [...]
#   df_completo_plataformas = <seu DF completo, sem deduplicar por UID>
#   col_plataforma = 'Platform Name'  # ajuste se necessário
# ================================================================

# -------- parâmetros do usuário --------
PLATAFORMAS_EXCLUIR
df_completo_plataformas = Base_BB_import
col_plataforma = 'Platform Name'

# -------- checagens explícitas (sem try/except silencioso) --------
if 'BB UID' not in df_completo_plataformas.columns:
    raise KeyError("O DF completo informado NÃO tem a coluna 'BB UID'.")
if col_plataforma not in df_completo_plataformas.columns:
    raise KeyError(f"O DF completo informado NÃO tem a coluna '{col_plataforma}'.")

# -------- mapeia UID -> plataformas (excluindo as indesejadas) --------
f_plat = df_completo_plataformas.copy()

# normaliza plataforma (string) e elimina vazios
f_plat[col_plataforma] = f_plat[col_plataforma].astype(str).str.strip()
f_plat = f_plat[f_plat[col_plataforma] != '']

# remove plataformas indesejadas
mask_keep = ~f_plat[col_plataforma].isin(PLATAFORMAS_EXCLUIR)
f_plat = f_plat[mask_keep]

uid_to_plats = (
    f_plat.groupby('BB UID', as_index=False)[col_plataforma]
          .agg(lambda s: ' | '.join(sorted(set(s))))
          .rename(columns={col_plataforma: 'Plataformas'})
)

# -------- obtém UIDs já casados para excluir da revisão --------
cpb_col = 'Nº CPB' if 'Nº CPB' in filmesBR_SAD_limpo.columns else 'N° CPB'
uids_casados = set(matches_all['UID'].unique())

# -------- função para gerar fila (filmes/séries estrangeiras) --------
def fila_revisao_estr(df_estr, categoria_label):
    if 'BB Countries' not in df_estr.columns:
        raise KeyError(f"O DF '{categoria_label}' não tem a coluna 'BB Countries'.")
    if 'BB Directors' not in df_estr.columns:
        raise KeyError(f"O DF '{categoria_label}' não tem a coluna 'BB Directors'.")

    # country vazio + diretor presente
    base = df_estr[
        (df_estr['BB Countries'].isna()) &
        (df_estr['BB Directors'].astype(str).str.strip() != '')
    ].copy()

    # remove os que já casaram
    base = base[~base['BB UID'].isin(uids_casados)].copy()

    # junta plataformas a partir do DF completo
    out = base.merge(uid_to_plats, on='BB UID', how='left')

    # mantemos apenas quem de fato tem plataforma (após exclusão)
    out = out[out['Plataformas'].notna() & (out['Plataformas'].str.strip() != '')].copy()

    # colunas úteis para revisão
    cols = [c for c in [
        'BB UID', 'BB Title', 'BB Original Title', 'BB Year',
        'BB Directors', 'BB Countries',
        'Plataformas'
    ] if c in out.columns]

    out = out[cols].sort_values(['Plataformas','BB Year','BB Title'], na_position='last')
    out['categoria'] = categoria_label
    return out

# -------- gerar filas para filmes/séries estrangeiras --------
revisao_filmes_EST  = fila_revisao_estr(filmesEstr_BB_limpo, 'filmes_EST')
revisao_series_EST  = fila_revisao_estr(seriesEstr_BB_limpo, 'series_EST')

print("Resumo das filas de revisão:")
print("filmes_EST:", len(revisao_filmes_EST))
print("series_EST:", len(revisao_series_EST))

# -------- exportar para Excel --------
arq_revisao = 'Revisao - estr sem country (por plataformas).xlsx'
with pd.ExcelWriter(arq_revisao, engine='openpyxl') as wr:
    revisao_filmes_EST.to_excel(wr, sheet_name='filmes_EST', index=False)
    revisao_series_EST.to_excel(wr, sheet_name='series_EST', index=False)
print(f"Arquivo salvo: {arq_revisao}")


Resumo das filas de revisão:
filmes_EST: 11880
series_EST: 763
Arquivo salvo: Revisao - estr sem country (por plataformas).xlsx


In [61]:
# ================================================================
# Listas de NÃO-MATCH para revisão
#  - Estrg (filmes/séries): por plataformas-alvo, diretor presente, não casados
#  - BR   (filmes/séries):  BB Countries contém "BR", não casados
#  Usa Base_BB (completo) para mapear UID -> plataformas e 1º deeplink
# ================================================================

PLATAFORMAS_ALVO = [
    'Filmicca','AmazôniaFLIX','TV Brasil Play','UOL Play',
    'Cine Humberto Mauro Mais','Reserva Imovision','Curta!On','Cinemateca Pernambucana',
]

def _uid_col(df):
    return 'BB UID' if 'BB UID' in df.columns else 'UID'

# ---------------- Checagens mínimas ----------------
for name, df in [
    ('Base_BB_import', Base_BB_import),
    ('filmesBR_BB_limpo', filmesBR_BB_limpo),
    ('seriesBR_BB_limpo', seriesBR_BB_limpo),
    ('filmesEstr_BB_limpo', filmesEstr_BB_limpo),
    ('seriesEstr_BB_limpo', seriesEstr_BB_limpo),
]:
    if _uid_col(df) not in df.columns:
        raise KeyError(f"{name} não tem coluna de UID ('BB UID' ou 'UID').")

if 'Platform Name' not in Base_BB_import.columns or 'Deeplink' not in Base_BB_import.columns:
    raise KeyError("Base_BB_import precisa de 'Platform Name' e 'Deeplink'.")

# ---------------- UIDs já casados (para excluir) ----------------
uids_casados = set(matches_all[_uid_col(matches_all)].unique())

# ---------------- Mapa UID -> plataformas-alvo + 1º deeplink ----------------
uid_full = _uid_col(Base_BB_import)
bb_plat_keep = Base_BB_import[[uid_full,'Platform Name','Deeplink']].copy()
bb_plat_keep = bb_plat_keep[bb_plat_keep['Platform Name'].isin(PLATAFORMAS_ALVO)]

uid_to_plats = (
    bb_plat_keep.sort_values(['Platform Name'])
    .groupby(uid_full, as_index=False)
    .agg({
        'Platform Name': lambda s: ' | '.join(sorted(set(s))),
        'Deeplink': 'first'
    })
    .rename(columns={'Platform Name': 'Plataformas', 'Deeplink': 'Deeplink_1'})
)

# ---------------- Funções canônicas para montar as listas ----------------
BASE_COLS = [
    'BB Title','BB Original Title','BB Year',
    'Seasons','Episodes','Season Numbers',
    'BB Directors','BB Countries',
    'BB Primary Company','BB Production Companies'
]

ORDER_COLS = ['Plataformas','BB Year','BB Title']  # para Estrg
ORDER_COLS_BR = ['BB Year','BB Title']            # para BR

def lista_estrg_por_plataforma(df_estr, categoria_label):
    uidc = _uid_col(df_estr)
    for c in BASE_COLS:
        if c not in df_estr.columns:
            raise KeyError(f"{categoria_label}: coluna '{c}' ausente.")

    base = df_estr.copy()

    # 1) não casados
    base = base[~base[uidc].isin(uids_casados)]

    # 2) DIRETOR OBRIGATÓRIO
    base = base[nonempty_str(base['BB Directors'])]

    # 3) junta plataformas-alvo + deeplink (Left Join; depois mantém só quem achou plataforma-alvo)
    out = base.merge(uid_to_plats, left_on=uidc, right_on=uid_full, how='left')
    out = out[nonempty_str(out['Plataformas'])].copy()

    cols = [uidc] + BASE_COLS + ['Plataformas','Deeplink_1']
    out = out[cols].sort_values(ORDER_COLS, na_position='last')
    out['categoria'] = categoria_label
    return out

def lista_br_sem_match(df_br, categoria_label):
    uidc = _uid_col(df_br)
    for c in BASE_COLS:
        if c not in df_br.columns:
            raise KeyError(f"{categoria_label}: coluna '{c}' ausente.")

    base = df_br.copy()

    # 1) não casados
    base = base[~base[uidc].isin(uids_casados)]

    # 2) Country contém "BR" (casos sem BR já foram para as listas Estrg)
    base = base[base['BB Countries'].astype(str).str.contains('BR', case=False, na=False)]

    out = base.merge(uid_to_plats, left_on=uidc, right_on=uid_full, how='left')
    cols = [uidc] + BASE_COLS + ['Plataformas','Deeplink_1']
    out = out[cols].sort_values(ORDER_COLS_BR, na_position='last')
    out['categoria'] = categoria_label
    return out

# ---------------- Materialização (4 listas) ----------------
estrg_filmes = lista_estrg_por_plataforma(filmesEstr_BB_limpo, 'Estrg Filmes')
estrg_series = lista_estrg_por_plataforma(seriesEstr_BB_limpo, 'Estrg Séries')
br_filmes    = lista_br_sem_match(filmesBR_BB_limpo, 'BR Filmes')
br_series    = lista_br_sem_match(seriesBR_BB_limpo, 'BR Séries')

# Diagnóstico rápido
print("Resumo (linhas):",
      f"Estrg Filmes={len(estrg_filmes)} | Estrg Séries={len(estrg_series)} | BR Filmes={len(br_filmes)} | BR Séries={len(br_series)}")

# ---------------- Export ----------------
arq_listas = "Revisao - NaoMatch (Estrg por plataforma + BR).xlsx"
with pd.ExcelWriter(arq_listas, engine='openpyxl') as wr:
    estrg_filmes.to_excel(wr, sheet_name='Estrg Filmes', index=False)
    estrg_series.to_excel(wr, sheet_name='Estrg Séries', index=False)
    br_filmes.to_excel(wr,    sheet_name='BR Filmes',    index=False)
    br_series.to_excel(wr,    sheet_name='BR Séries',    index=False)
print(f"Arquivo salvo: {arq_listas}")

Resumo (linhas): Estrg Filmes=435 | Estrg Séries=77 | BR Filmes=381 | BR Séries=264
Arquivo salvo: Revisao - NaoMatch (Estrg por plataforma + BR).xlsx


## 5) Criação da listas de buscas manual

### 5.1) Criação das listas coletiva

In [62]:
# ================================================================
# Listas finais de NÃO-MATCH para revisão manual
#  - Estrg (filmes/séries): sem CPB, país vazio, diretor presente,
#    e presentes nas plataformas foco (whitelist)
#  - BR   (filmes/séries): sem CPB e BB Countries contendo "BR"
#  Join por BB UID na Base_BB_import para trazer Plataformas e Deeplink
# ================================================================

PLATAFORMAS_FOCO = [
    'Filmicca',
    'AmazôniaFLIX',
    'TV Brasil Play',
    'UOL Play',
    'Cine Humberto Mauro Mais',
    'Reserva Imovision',
    'Curta!On',
    'Cinemateca Pernambucana',
]

def _uidcol(df):
    return 'BB UID' if 'BB UID' in df.columns else 'UID'

# UIDs já casados (para excluir)
uids_casados = set(matches_all[_uidcol(matches_all)].unique())

# Mapa UID -> plataformas whitelist + 1º deeplink
uid_full = _uidcol(Base_BB_import)
plat_map = (
    Base_BB_import[[uid_full, 'Platform Name', 'Deeplink']]
    .query("`Platform Name` in @PLATAFORMAS_FOCO")
    .sort_values(['Platform Name'])
    .groupby(uid_full, as_index=False)
    .agg({'Platform Name': lambda s: ' | '.join(sorted(set(s))),
          'Deeplink': 'first'})
    .rename(columns={'Platform Name':'Plataformas', 'Deeplink':'Deeplink_1'})
)

def _montar_lista(df_bb,
                  precisa_br=None,
                  precisa_diretor=False,
                  filtrar_plataforma=False,
                  categoria=''):
    uid = _uidcol(df_bb)

    base = df_bb.copy()
    base = base[~base[uid].isin(uids_casados)]

    if precisa_br is True:
        base = base[base['BB Countries'].astype(str).str.contains('BR', case=False, na=False)]
    elif precisa_br is False:
        base = base[~base['BB Countries'].astype(str).str.contains('BR', case=False, na=False)]
        base = base[base['BB Countries'].isna() | (base['BB Countries'].astype(str).str.strip() == '')]

    # ✅ CORREÇÃO AQUI - use nonempty_str()
    if precisa_diretor:
        base = base[nonempty_str(base['BB Directors'])]  # ← MUDANÇA

    if filtrar_plataforma:
        base = base.merge(plat_map, left_on=uid, right_on=uid_full, how='inner')
    else:
        base = base.merge(plat_map, left_on=uid, right_on=uid_full, how='left')

    final_cols = [
        uid, 'Platform Title', 'BB Original Title', 'BB Title', 'BB Year',
        'Seasons', 'Episodes', 'Season Numbers',
        'BB Directors', 'BB Primary Company', 'BB Production Companies',
        'BB Countries', 'Plataformas', 'Deeplink_1'
    ]
    final_cols = [c for c in final_cols if c in base.columns]

    out = base[final_cols].sort_values(['Plataformas','BB Year','BB Title'], na_position='last')
    out['categoria'] = categoria

    for c in ['Responsável', 'Nº CPB encontrado', 'Título encontrado', 'Observação']:
        out[c] = pd.NA
    return out

# -------- 4 listas finais --------
filmes_ESTR = _montar_lista(filmesEstr_BB_limpo, precisa_br=False, precisa_diretor=True,
                            filtrar_plataforma=True,  categoria='filmes_ESTR')
series_ESTR = _montar_lista(seriesEstr_BB_limpo, precisa_br=False, precisa_diretor=True,
                            filtrar_plataforma=True,  categoria='series_ESTR')
filmes_BR   = _montar_lista(filmesBR_BB_limpo,   precisa_br=True,  precisa_diretor=False,
                            filtrar_plataforma=False, categoria='filmes_BR')
series_BR   = _montar_lista(seriesBR_BB_limpo,   precisa_br=True,  precisa_diretor=False,
                            filtrar_plataforma=False, categoria='series_BR')

# Sanidade: interseção deve ser 0
u_estr = set(filmes_ESTR[_uidcol(filmes_ESTR)]) | set(series_ESTR[_uidcol(series_ESTR)])
u_br   = set(filmes_BR[_uidcol(filmes_BR)])     | set(series_BR[_uidcol(series_BR)])
print("Interseção (UIDs em ESTR e BR):", len(u_estr & u_br))



Interseção (UIDs em ESTR e BR): 0


In [63]:
# Exporta
arq = "Revisao - NaoMatch (Estrg por plataforma + BR) — FINAL.xlsx"
with pd.ExcelWriter(arq, engine='openpyxl') as wr:
    filmes_BR.to_excel(wr,   sheet_name='filmes_BR',   index=False)
    series_BR.to_excel(wr,   sheet_name='series_BR',   index=False)
    filmes_ESTR.to_excel(wr, sheet_name='filmes_ESTR', index=False)
    series_ESTR.to_excel(wr, sheet_name='series_ESTR', index=False)
    
    # Lista completa empilhada
    lista_completa = pd.concat([filmes_BR, series_BR, filmes_ESTR, series_ESTR], ignore_index=True)
    lista_completa.to_excel(wr, sheet_name='lista_completa', index=False)
    
print("Arquivo salvo:", arq)

Arquivo salvo: Revisao - NaoMatch (Estrg por plataforma + BR) — FINAL.xlsx
